In [1]:
%pip install mplfinance

Defaulting to user installation because normal site-packages is not writeable
  Using cached mplfinance-0.12.9b1-py3-none-any.whl (70 kB)

[notice] A new release of pip available: 22.1.2 -> 22.2.2
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install stockstats

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip available: 22.1.2 -> 22.2.2
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import matplotlib.pyplot as plt
import mplfinance
import numpy as np

In [4]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt

import datetime
import yfinance as yf
import time
import stockstats
import chart_studio.plotly as py
import plotly.graph_objs as go
import plotly.figure_factory as ff
from plotly.offline import iplot, init_notebook_mode
from collections import OrderedDict
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix

In [2]:
# df_test = pd.read_csv("../Data/ADANIPOWER__EQ__NSE__NSE__MINUTE.csv")
# df_test = df_test[["open","close"]]

In [3]:
# df_test['MA100'] = df_test.rolling(window=100,min_periods=1 ).mean()['close']
# df_test['MA200'] = df_test.rolling(window=200,min_periods=1 ).mean()['close']
# df_test['MA50'] = df_test.rolling(window=50,min_periods=1 ).mean()['close']
# df_test['MA5'] = df_test.rolling(window=5,min_periods=1 ).mean()['close']
# df_test['C_BodyHi'] = df_test[['close','open']].max(axis=1)
# df_test['C_BodyLo'] = df_test[['close','open']].min(axis=1)
# df_test['C_Body'] = df_test['C_BodyHi'] - df_test['C_BodyLo']
# df_test['C_BodyAvg'] = pd.Series.ewm(df_test['C_Body'], span=14).mean()

In [1]:
# df_test.head(50)

A bearish abandoned baby is a specialized candlestick pattern consisting of three candles, one with rising prices, a second with holding prices, and a third with falling prices.

In [5]:
def AbandonedBabyBear(df1):
    df1 = df.copy()

    #moving average calculated over 100 200 and 50 days
    df1['MA100'] = df.rolling(window=100,min_periods=1 ).mean()['close']
    df1['MA200'] = df.rolling(window=200,min_periods=1 ).mean()['close']
    df1['MA50'] = df.rolling(window=50,min_periods=1 ).mean()['close']
    C_DownTrend = True
    C_UpTrend = True

    #marking C_DownTrend and C_UpTrend as true or false according to respective conditions and patterns
    df1['C_DownTrend'] = np.where((  (df1['close'] < df1['MA50']) & (df1['MA50'] < df1['MA200']) ), True, False)
    df1['C_UpTrend'] = np.where(( (df1['close'] > df1['MA50'])& (df1['MA50'] > df1['MA200']) ), True, False)
    C_Len = 14 #ema depth for bodyAvg
    C_ShadowPercent = 5.0 # size of shadows
    C_ShadowEqualsPercent = 100.0
    C_DojiBodyPercent = 5.0
    C_Factor = 2.0 # shows the number of times the shadow dominates the candlestick body
    
    #finding body high and low and body length from closing and opening price of the day
    df1['C_BodyHi'] = df1[['close','open']].max(axis=1)
    df1['C_BodyLo'] = df1[['close','open']].min(axis=1)
    df1['C_Body'] = df1['C_BodyHi'] - df1['C_BodyLo']
    
    df1['C_BodyAvg'] = pd.Series.ewm(df1['C_Body'], span=C_Len).mean()
    
    #setting C_SmallBody as 1 if body average is > the body length and 0 if body average is <= to body length
    df1.loc[df1['C_BodyAvg']> df1['C_Body']  , 'C_SmallBody'] = '1'
    df1.loc[df1['C_BodyAvg']<= df1['C_Body']  , 'C_SmallBody'] = '0'

    #setting C_LongBody as 1 if body average is < the body length and 0 if body average is >= to body length
    df1.loc[df1['C_BodyAvg']< df1['C_Body']  , 'C_LongBody'] = '1'
    df1.loc[df1['C_BodyAvg']>= df1['C_Body']  , 'C_LongBody'] = '0'
    
    #calculating upshadow as high - body high and down shadow as body low - low
    df1['C_UpShadow'] = df1['high'] - df1['C_BodyHi']
    df1['C_DnShadow'] = df1['C_BodyLo'] - df1['low']

    #setting C_HasUpShadow as 0 or 1 indicating presence of up shadow
    df1['C_HasUpShadow'] = np.where((df1.C_UpShadow  > (C_ShadowPercent / (100*df1.C_Body))  ),
                                                     "1", "0")
    #setting C_HasDnShadow as 0 or 1 indicating presence of down shadow                                                 
    df1['C_HasDnShadow'] = np.where((df1.C_DnShadow  > (C_ShadowPercent / (100*df1.C_Body))  ),
                                                     "1", "0")
    
    #white body reprecents traditional green candlestick and black represents red
    df1['C_WhiteBody'] = np.where((df1.open  < df1.close ),True, False)
    df1['C_BlackBody'] = np.where((df1.open  > df1.close ),True, False)
    

    df1['C_Range'] = df1['high']-df1['low']
    df1['C_BodyMiddle']= df1['C_Body'] / (2 + df1['C_BodyLo'])
    
    #findng if upshadow is equal to dowm shadow 
    df1['C_ShadowEquals'] = np.where(  ((df1['C_UpShadow'] == df1['C_DnShadow']) | 
                                 (( abs(df1['C_UpShadow'] - df1['C_DnShadow']) / (df1['C_DnShadow'] * 100) ) )  < 
                                 C_ShadowEqualsPercent) & (((abs(df1['C_DnShadow'] - df1['C_UpShadow']) / (df1['C_UpShadow'] * 100))
                                 < C_ShadowEqualsPercent))  , True, False)
    
    #checking if body range is <= 20 percent of body range of candlestick and setting doji body as true or false accordingly
    df1['C_IsDojiBody'] = np.where(( (df1['C_Range'] > 0)  & (df1['C_Body'] <= (df1['C_Range'] * (C_DojiBodyPercent / 100))) ), True, False)
    
    
    df1['C_Doji'] = df1['C_IsDojiBody'] & df1['C_ShadowEquals']
    
    #checking if it is an abandoned baby bearish pattern and marking as 1 if yes and 0 if no
    df1['C_AbandonedBabyBearish'] = np.where( ( ( df1['C_UpTrend'].shift(2)  & df1['C_WhiteBody'].shift(2) & 
                                             df1['C_IsDojiBody'].shift(1) &  (df1['high'].shift(2) <df1['low'].shift(1)) &
                                             df1['C_BlackBody'] & (df1['low'].shift(1) > df1['high'] )   ) )    , "1", "0" )
    
    return df1

It forms in a downtrend and is composed of three price bars. The first is a large down candle, followed by a doji candle that gaps below the first candle. The next candle opens higher than the doji and moves aggressively to the upside.

In [17]:
def AbandonedBabyBull(df1):
    df1 = df.copy()

    #moving average calculated over 100 200 and 50 days
    df1['MA100'] = df.rolling(window=100,min_periods=1 ).mean()['close']
    df1['MA200'] = df.rolling(window=200,min_periods=1 ).mean()['close']
    df1['MA50'] = df.rolling(window=50,min_periods=1 ).mean()['close']
    C_DownTrend = True
    C_UpTrend = True

    #marking C_DownTrend and C_UpTrend as true or false according to respective conditions and patterns
    df1['C_DownTrend'] = np.where((  (df1['close'] < df1['MA50']) & (df1['MA50'] < df1['MA200']) ), True, False)
    df1['C_UpTrend'] = np.where(( (df1['close'] > df1['MA50'])& (df1['MA50'] > df1['MA200']) ), True, False)
    C_Len = 14 #ema depth for bodyAvg
    C_ShadowPercent = 5.0 # size of shadows
    C_ShadowEqualsPercent = 100.0
    C_DojiBodyPercent = 5.0
    C_Factor = 2.0 # shows the number of times the shadow dominates the candlestick body
    
    #finding body high and low and body length from closing and opening price of the day
    df1['C_BodyHi'] = df1[['close','open']].max(axis=1)
    df1['C_BodyLo'] = df1[['close','open']].min(axis=1)
    df1['C_Body'] = df1['C_BodyHi'] - df1['C_BodyLo']
    
    df1['C_BodyAvg'] = pd.Series.ewm(df1['C_Body'], span=C_Len).mean()
    
    #setting C_SmallBody as 1 if body average is > the body length and 0 if body average is <= to body length
    df1.loc[df1['C_BodyAvg']> df1['C_Body']  , 'C_SmallBody'] = True
    df1.loc[df1['C_BodyAvg']<= df1['C_Body']  , 'C_SmallBody'] = False

    #setting C_LongBody as 1 if body average is < the body length and 0 if body average is >= to body length
    df1.loc[df1['C_BodyAvg']< df1['C_Body']  , 'C_LongBody'] = True
    df1.loc[df1['C_BodyAvg']>= df1['C_Body']  , 'C_LongBody'] = False
    
    

    df1['C_UpShadow'] = df1['high'] - df1['C_BodyHi']
    df1['C_DnShadow'] = df1['C_BodyLo'] - df1['low']
    #setting C_HasUpShadow as 0 or 1 indicating presence of up shadow
    df1['C_HasUpShadow'] = np.where((df1.C_UpShadow  > (C_ShadowPercent / (100*df1.C_Body))  ),
                                                     True, False)
    #setting C_HasDnShadow as 0 or 1 indicating presence of down shadow
    df1['C_HasDnShadow'] = np.where((df1.C_DnShadow  > (C_ShadowPercent / (100*df1.C_Body))  ),
                                                     True, False)
    
    #white body reprecents traditional green candlestick and black represents red
    df1['C_WhiteBody'] = np.where((df1.open  < df1.close ),True, False)
    df1['C_BlackBody'] = np.where((df1.open  > df1.close ),True, False)
    
    df1['C_Range'] = df1['high']-df1['low']
    
    df1['C_IsInsideBar'] = np.where( ( ( df1['C_BodyHi'].shift(1) > df1['C_BodyHi'])  & 
                                      (df1['C_BodyLo'].shift(1) < df1['C_BodyLo'] ) ) , True, False)
    
    df1['C_BodyMiddle']= df1['C_Body'] / (2 + df1['C_BodyLo'])
    
    #findng if upshadow is equal to dowm shadow
    df1['C_ShadowEquals'] = np.where(  ((df1['C_UpShadow'] == df1['C_DnShadow']) | 
                                 (( abs(df1['C_UpShadow'] - df1['C_DnShadow']) / (df1['C_DnShadow'] * 100) ) )  < 
                                 C_ShadowEqualsPercent) & (((abs(df1['C_DnShadow'] - df1['C_UpShadow']) / (df1['C_UpShadow'] * 100))
                                 < C_ShadowEqualsPercent))  , True, False)
    
    #checking if body range is <= 20 percent of body range of candlestick and setting doji body as true or false accordingly
    df1['C_IsDojiBody'] = np.where(( (df1['C_Range'] > 0)  & (df1['C_Body'] <= (df1['C_Range'] * (C_DojiBodyPercent / 100))) ), True, False)
    
    df1['C_Doji'] = df1['C_IsDojiBody'] & df1['C_ShadowEquals']

    #checking if it is an abandoned baby bullish pattern and marking as 1 if yes and 0 if no
    df1['C_AbandonedBabyBullish'] = np.where( ( ( df1['C_DownTrend'].shift(2)  & df1['C_BlackBody'].shift(2) & 
                                             df1['C_IsDojiBody'].shift(1) &  (df1['low'].shift(2) > df1['high'].shift(1)) &
                                             df1['C_WhiteBody'] & (df1['high'].shift(1) < df1['low'] )   ) )    , "1", "0" )
    
    return df1

As with all candlestick patterns, four data points are used in their construction. The open is near the top of the pattern as is the close. Because the two datapoints are close, the real body is small. The real body of the hanging man can be black or white, but it must be small. The hanging man will have a long lower shadow which is two or three times the length of the real body. The low and the high of the candle (in our case, trading day) is at extreme ends of the price range during the trading day. There may or may not be an upper shadow. If there is an upper shadow it will be small. but usually the upper shadow is non existent which means that that open or close and the high are the same.

In [18]:
def HangingManBear(df):
    df1 = df.copy()
    
    #moving average calculated over 100 200 and 50 days
    df1['MA100'] = df.rolling(window=100,min_periods=1 ).mean()['close']
    df1['MA200'] = df.rolling(window=200,min_periods=1 ).mean()['close']
    df1['MA50'] = df.rolling(window=50,min_periods=1 ).mean()['close']
    C_DownTrend = True
    C_UpTrend = True

    #marking C_DownTrend and C_UpTrend as true or false according to respective conditions and patterns
    df1['C_DownTrend'] = np.where((  (df1['close'] < df1['MA50']) & (df1['MA50'] < df1['MA200']) ), True, False)
    df1['C_UpTrend'] = np.where(( (df1['close'] > df1['MA50'])& (df1['MA50'] > df1['MA200']) ), True, False)
    C_Len = 14 #ema depth for bodyAvg
    C_ShadowPercent = 5.0 # size of shadows
    C_ShadowEqualsPercent = 100.0
    C_DojiBodyPercent = 5.0
    C_Factor = 2.0 # shows the number of times the shadow dominates the candlestick body
    
    #finding body high and low and body length from closing and opening price of the day
    df1['C_BodyHi'] = df1[['close','open']].max(axis=1)
    df1['C_BodyLo'] = df1[['close','open']].min(axis=1)
    df1['C_Body'] = df1['C_BodyHi'] - df1['C_BodyLo']
    
    df1['C_BodyAvg'] = pd.Series.ewm(df1['C_Body'], span=C_Len).mean()
    
    #setting C_SmallBody as 1 if body average is > the body length and 0 if body average is <= to body length
    df1.loc[df1['C_BodyAvg']> df1['C_Body']  , 'C_SmallBody'] = True
    df1.loc[df1['C_BodyAvg']<= df1['C_Body']  , 'C_SmallBody'] = False

    #setting C_LongBody as 1 if body average is < the body length and 0 if body average is >= to body length
    df1.loc[df1['C_BodyAvg']< df1['C_Body']  , 'C_LongBody'] = True
    df1.loc[df1['C_BodyAvg']>= df1['C_Body']  , 'C_LongBody'] = False
    
    #calculating upshadow as high - body high and down shadow as body low - low
    df1['C_UpShadow'] = df1['high'] - df1['C_BodyHi']
    df1['C_DnShadow'] = df1['C_BodyLo'] - df1['low']

    #setting C_HasUpShadow as 0 or 1 indicating presence of up shadow
    df1['C_HasUpShadow'] = np.where((df1.C_UpShadow  > (C_ShadowPercent / (100*df1.C_Body))  ),
                                                     True, False)
    #setting C_HasDnShadow as 0 or 1 indicating presence of down shadow
    df1['C_HasDnShadow'] = np.where((df1.C_DnShadow  > (C_ShadowPercent / (100*df1.C_Body))  ),
                                                     True, False)
    
    #white body reprecents traditional green candlestick and black represents red
    df1['C_WhiteBody'] = np.where((df1.open  < df1.close ),True, False)
    df1['C_BlackBody'] = np.where((df1.open  > df1.close ),True, False)
    
    df1['C_Range'] = df1['high']-df1['low']
    df1['C_BodyMiddle']= df1['C_Body'] / (2 + df1['C_BodyLo'])
    
    #findng if upshadow is equal to dowm shadow
    df1['C_ShadowEquals'] = np.where(  ((df1['C_UpShadow'] == df1['C_DnShadow']) | 
                                 (( abs(df1['C_UpShadow'] - df1['C_DnShadow']) / (df1['C_DnShadow'] * 100) ) )  < 
                                 C_ShadowEqualsPercent) & (((abs(df1['C_DnShadow'] - df1['C_UpShadow']) / (df1['C_UpShadow'] * 100))
                                 < C_ShadowEqualsPercent))  , True, False)
    
    #checking if body range is <= 20 percent of body range of candlestick and setting doji body as true or false accordingly
    df1['C_IsDojiBody'] = np.where(( (df1['C_Range'] > 0)  & (df1['C_Body'] <= (df1['C_Range'] * (C_DojiBodyPercent / 100))) ), True, False)
    
    df1['C_Doji'] = df1['C_IsDojiBody'] & df1['C_ShadowEquals']
    df1['hl2'] = (df['high'] + df1['low'])/2
    
    #checking if it is an Hanging Man Bearish pattern and marking as 1 if yes and 0 if no
    df1['C_HangingManBearish'] = np.where( (( df1['C_UpTrend']  & df1['C_SmallBody'] & 
                                             df1['C_Body'] > 0 &  (df1['hl2'] < df1['C_BodyLo']) &
                                             (df1['C_DnShadow'] >= (C_Factor * df1['C_Body'])) & (~ df1['C_HasUpShadow'])  ) ), "1", "0" )
    
    return df1


The Bearish Doji Star is a bearish reversal pattern represented by two candles. During an uptrend, the first candle is increasing and has a long body. It is followed by a Doji that opens and closes above the previous candle.

In [19]:
def DojiStarBear(df):
    df1 = df.copy()
    #moving average calculated over 100 200 and 50 days
    df1['MA100'] = df.rolling(window=100,min_periods=1 ).mean()['close']
    df1['MA200'] = df.rolling(window=200,min_periods=1 ).mean()['close']
    df1['MA50'] = df.rolling(window=50,min_periods=1 ).mean()['close']
    C_DownTrend = True
    C_UpTrend = True
    #marking C_DownTrend and C_UpTrend as true or false according to respective conditions and patterns
    df1['C_DownTrend'] = np.where((  (df1['close'] < df1['MA50']) & (df1['MA50'] < df1['MA200']) ), True, False)
    df1['C_UpTrend'] = np.where(( (df1['close'] > df1['MA50'])& (df1['MA50'] > df1['MA200']) ), True, False)
    C_Len = 14 #ema depth for bodyAvg
    C_ShadowPercent = 5.0 # size of shadows
    C_ShadowEqualsPercent = 100.0
    C_DojiBodyPercent = 5.0
    C_Factor = 2.0 # shows the number of times the shadow dominates the candlestick body
    #finding body high and low and body length from closing and opening price of the day
    df1['C_BodyHi'] = df1[['close','open']].max(axis=1)
    df1['C_BodyLo'] = df1[['close','open']].min(axis=1)
    df1['C_Body'] = df1['C_BodyHi'] - df1['C_BodyLo']
    
    df1['C_BodyAvg'] = pd.Series.ewm(df1['C_Body'], span=C_Len).mean()
        #setting C_SmallBody as 1 if body average is > the body length and 0 if body average is <= to body length
    
    df1.loc[df1['C_BodyAvg']> df1['C_Body']  , 'C_SmallBody'] = True
    df1.loc[df1['C_BodyAvg']<= df1['C_Body']  , 'C_SmallBody'] = False

    #setting C_LongBody as 1 if body average is < the body length and 0 if body average is >= to body length
    df1.loc[df1['C_BodyAvg']< df1['C_Body']  , 'C_LongBody'] = True
    df1.loc[df1['C_BodyAvg']>= df1['C_Body']  , 'C_LongBody'] = False
    
    
    #calculating upshadow as high - body high and down shadow as body low - low
    df1['C_UpShadow'] = df1['high'] - df1['C_BodyHi']
    df1['C_DnShadow'] = df1['C_BodyLo'] - df1['low']
        #setting C_HasUpShadow as 0 or 1 indicating presence of up shadow
    df1['C_HasUpShadow'] = np.where((df1.C_UpShadow  > (C_ShadowPercent / (100*df1.C_Body))  ),
                                                     True, False)

         #setting C_HasDnShadow as 0 or 1 indicating presence of down shadow                                                
    df1['C_HasDnShadow'] = np.where((df1.C_DnShadow  > (C_ShadowPercent / (100*df1.C_Body))  ),
                                                     True, False)
    
    #white body reprecents traditional green candlestick and black represents red
    df1['C_WhiteBody'] = np.where((df1.open  < df1.close ),True, False)
    df1['C_BlackBody'] = np.where((df1.open  > df1.close ),True, False)
    
    df1['C_Range'] = df1['high']-df1['low']
    df1['C_BodyMiddle']= df1['C_Body'] / (2 + df1['C_BodyLo'])
    
    #findng if upshadow is equal to dowm shadow
    df1['C_ShadowEquals'] = np.where(  ((df1['C_UpShadow'] == df1['C_DnShadow']) | 
                                 (( abs(df1['C_UpShadow'] - df1['C_DnShadow']) / (df1['C_DnShadow'] * 100) ) )  < 
                                 C_ShadowEqualsPercent) & (((abs(df1['C_DnShadow'] - df1['C_UpShadow']) / (df1['C_UpShadow'] * 100))
                                 < C_ShadowEqualsPercent))  , True, False)
    
        #checking if body range is <= 20 percent of body range of candlestick and setting doji body as true or false accordingly
    df1['C_IsDojiBody'] = np.where(( (df1['C_Range'] > 0)  & (df1['C_Body'] <= (df1['C_Range'] * (C_DojiBodyPercent / 100))) ), True, False)
    
    df1['C_Doji'] = df1['C_IsDojiBody'] & df1['C_ShadowEquals']
    
    df1['hl2'] = (df['high'] + df1['low'])/2
    
    #checking if it is an Doji star Bearish pattern and marking as 1 if yes and 0 if no
    df1['C_DojiStarBearish'] = np.where( (( df1['C_UpTrend']  & df1['C_WhiteBody'].shift(1) & 
                                             df1['C_LongBody'].shift(1) &  df1['C_IsDojiBody'] &
                                             (df1['C_BodyLo'] >  df1['C_BodyHi'].shift(1) )  ) ), "1", "0" )
    
    return df1


Doji star bullish :
This is a bullish reversal candlestick pattern that is found in a downtrend and consists of two candles. First comes a long red candle, followed by a Doji candle (except 4-Price Doji) that opens below the body of the first one, creating a gap.


How to identify the bullish Doji Star candlestick? 
Look for a normal red candlestick on the very bottom of the charts on the first day. If confirms the prevalent downtrend and also illustrates that price closed lesser than the opening price. 
Next, look for a small Doji on the second day showing that there is very minimal or no gap at all between opening and closing prices. 
Now look for the third candlestick showing a gap up opening. 

In [20]:
def DojiStarBull(df):
    df1 = df.copy()
    #moving average calculated over 100 200 and 50 days
    df1['MA100'] = df.rolling(window=100,min_periods=1 ).mean()['close']
    df1['MA200'] = df.rolling(window=200,min_periods=1 ).mean()['close']
    df1['MA50'] = df.rolling(window=50,min_periods=1 ).mean()['close']
    C_DownTrend = True
    C_UpTrend = True

    #marking C_DownTrend and C_UpTrend as true or false according to respective conditions and patterns
    df1['C_DownTrend'] = np.where((  (df1['close'] < df1['MA50']) & (df1['MA50'] < df1['MA200']) ), True, False)
    df1['C_UpTrend'] = np.where(( (df1['close'] > df1['MA50'])& (df1['MA50'] > df1['MA200']) ), True, False)
    C_Len = 14 #ema depth for bodyAvg
    C_ShadowPercent = 5.0 # size of shadows
    C_ShadowEqualsPercent = 100.0
    C_DojiBodyPercent = 5.0
    C_Factor = 2.0 # shows the number of times the shadow dominates the candlestick body

    #finding body high and low and body length from closing and opening price of the day
    df1['C_BodyHi'] = df1[['close','open']].max(axis=1)
    df1['C_BodyLo'] = df1[['close','open']].min(axis=1)
    df1['C_Body'] = df1['C_BodyHi'] - df1['C_BodyLo']
    
    df1['C_BodyAvg'] = pd.Series.ewm(df1['C_Body'], span=C_Len).mean()
    
    #setting C_SmallBody as 1 if body average is > the body length and 0 if body average is <= to body length
    df1.loc[df1['C_BodyAvg']> df1['C_Body']  , 'C_SmallBody'] = True
    df1.loc[df1['C_BodyAvg']<= df1['C_Body']  , 'C_SmallBody'] = False

   #setting C_LongBody as 1 if body average is < the body length and 0 if body average is >= to body length
    df1.loc[df1['C_BodyAvg']< df1['C_Body']  , 'C_LongBody'] = True
    df1.loc[df1['C_BodyAvg']>= df1['C_Body']  , 'C_LongBody'] = False
    
    #calculating upshadow as high - body high and down shadow as body low - low
    df1['C_UpShadow'] = df1['high'] - df1['C_BodyHi']
     #setting C_HasDnShadow as 0 or 1 indicating presence of down shadow
    df1['C_DnShadow'] = df1['C_BodyLo'] - df1['low']

    #setting C_HasUpShadow as 0 or 1 indicating presence of up shadow
    df1['C_HasUpShadow'] = np.where((df1.C_UpShadow  > (C_ShadowPercent / (100*df1.C_Body))  ),
                                                     True, False)
    df1['C_HasDnShadow'] = np.where((df1.C_DnShadow  > (C_ShadowPercent / (100*df1.C_Body))  ),
                                                     True, False)
    #white body reprecents traditional green candlestick and black represents red
    df1['C_WhiteBody'] = np.where((df1.open  < df1.close ),True, False)
    df1['C_BlackBody'] = np.where((df1.open  > df1.close ),True, False)
    
    df1['C_Range'] = df1['high']-df1['low']
    df1['C_BodyMiddle']= df1['C_Body'] / (2 + df1['C_BodyLo'])
    
    #findng if upshadow is equal to dowm shadow
    df1['C_ShadowEquals'] = np.where(  ((df1['C_UpShadow'] == df1['C_DnShadow']) | 
                                 (( abs(df1['C_UpShadow'] - df1['C_DnShadow']) / (df1['C_DnShadow'] * 100) ) )  < 
                                 C_ShadowEqualsPercent) & (((abs(df1['C_DnShadow'] - df1['C_UpShadow']) / (df1['C_UpShadow'] * 100))
                                 < C_ShadowEqualsPercent))  , True, False)
    
    #checking if body range is <= 20 percent of body range of candlestick and setting doji body as true or false accordingly
    df1['C_IsDojiBody'] = np.where(( (df1['C_Range'] > 0)  & (df1['C_Body'] <= (df1['C_Range'] * (C_DojiBodyPercent / 100))) ), True, False)
    
    df1['C_Doji'] = df1['C_IsDojiBody'] & df1['C_ShadowEquals']
    
    df1['hl2'] = (df['high'] + df1['low'])/2
    
     #checking if it is an doji star bullish pattern and marking as 1 if yes and 0 if no
    df1['C_DojiStarBullish'] = np.where( (( df1['C_DownTrend']  & df1['C_BlackBody'].shift(1) & 
                                             df1['C_LongBody'].shift(1) &  df1['C_IsDojiBody'] &
                                             (df1['C_BodyLo'].shift(1) >  df1['C_BodyHi'])  ) ), "1", "0" )
    
    return df1


Dark Cloud Cover is a bearish reversal candlestick pattern where a down candle (typically black or red) opens above the close of the prior up candle (typically white or green), and then closes below the midpoint of the up candle.

The five criteria for the Dark Cloud Cover pattern are:

An existing bullish uptrend.
An up (bullish) candle within that uptrend.
A gap up on the following day.
The gap up turns into a down (bearish) candle.
The bearish candle closes below the midpoint of the previous bullish candle.


In [21]:
def DarkCloudCover(df):
    df1 = df.copy()
    
    #moving average calculated over 100 200 and 50 days
    df1['MA100'] = df.rolling(window=100,min_periods=1 ).mean()['close']
    df1['MA200'] = df.rolling(window=200,min_periods=1 ).mean()['close']
    df1['MA50'] = df.rolling(window=50,min_periods=1 ).mean()['close']
    C_DownTrend = True
    C_UpTrend = True
    #marking C_DownTrend and C_UpTrend as true or false according to respective conditions and patterns
    df1['C_DownTrend'] = np.where((  (df1['close'] < df1['MA50']) & (df1['MA50'] < df1['MA200']) ), True, False)
    df1['C_UpTrend'] = np.where(( (df1['close'] > df1['MA50'])& (df1['MA50'] > df1['MA200']) ), True, False)
    C_Len = 14 #ema depth for bodyAvg
    C_ShadowPercent = 5.0 # size of shadows
    C_ShadowEqualsPercent = 100.0
    C_DojiBodyPercent = 5.0
    C_Factor = 2.0 # shows the number of times the shadow dominates the candlestick body
     #finding body high and low and body length from closing and opening price of the day
    df1['C_BodyHi'] = df1[['close','open']].max(axis=1)
    df1['C_BodyLo'] = df1[['close','open']].min(axis=1)
    df1['C_Body'] = df1['C_BodyHi'] - df1['C_BodyLo']
    
    df1['C_BodyAvg'] = pd.Series.ewm(df1['C_Body'], span=C_Len).mean()
     #setting C_SmallBody as 1 if body average is > the body length and 0 if body average is <= to body length
    
    df1.loc[df1['C_BodyAvg']> df1['C_Body']  , 'C_SmallBody'] = True
    df1.loc[df1['C_BodyAvg']<= df1['C_Body']  , 'C_SmallBody'] = False
   #setting C_LongBody as 1 if body average is < the body length and 0 if body average is >= to body length
    df1.loc[df1['C_BodyAvg']< df1['C_Body']  , 'C_LongBody'] = True
    df1.loc[df1['C_BodyAvg']>= df1['C_Body']  , 'C_LongBody'] = False
    
    #calculating upshadow as high - body high and down shadow as body low - low
    df1['C_UpShadow'] = df1['high'] - df1['C_BodyHi']
    df1['C_DnShadow'] = df1['C_BodyLo'] - df1['low']
     #setting C_HasUpShadow as 0 or 1 indicating presence of up shadow
    df1['C_HasUpShadow'] = np.where((df1.C_UpShadow  > (C_ShadowPercent / (100*df1.C_Body))  ),
                                                     True, False)
      #setting C_HasDnShadow as 0 or 1 indicating presence of down shadow                                                                               
    df1['C_HasDnShadow'] = np.where((df1.C_DnShadow  > (C_ShadowPercent / (100*df1.C_Body))  ),
                                                     True, False)
    
    #white body reprecents traditional green candlestick and black represents red
    df1['C_WhiteBody'] = np.where((df1.open  < df1.close ),True, False)
    df1['C_BlackBody'] = np.where((df1.open  > df1.close ),True, False)
    
    df1['C_Range'] = df1['high']-df1['low']
    df1['C_BodyMiddle']= df1['C_Body'] / (2 + df1['C_BodyLo'])
    
     #findng if upshadow is equal to dowm shadow
    df1['C_ShadowEquals'] = np.where(  ((df1['C_UpShadow'] == df1['C_DnShadow']) | 
                                 (( abs(df1['C_UpShadow'] - df1['C_DnShadow']) / (df1['C_DnShadow'] * 100) ) )  < 
                                 C_ShadowEqualsPercent) & (((abs(df1['C_DnShadow'] - df1['C_UpShadow']) / (df1['C_UpShadow'] * 100))
                                 < C_ShadowEqualsPercent))  , True, False)
    
     #checking if body range is <= 20 percent of body range of candlestick and setting doji body as true or false accordingly
    df1['C_IsDojiBody'] = np.where(( (df1['C_Range'] > 0)  & (df1['C_Body'] <= (df1['C_Range'] * (C_DojiBodyPercent / 100))) ), True, False)
    
    df1['C_Doji'] = df1['C_IsDojiBody'] & df1['C_ShadowEquals']
    
    
    df1['hl2'] = (df['high'] + df1['low'])/2
    
    #checking if it is an Dark cloud cover Bearish pattern and marking as 1 if yes and 0 if no
    df1['C_DarkCloudCoverBearish'] = np.where( (  ( df1['C_UpTrend'].shift(1)  & df1['C_WhiteBody'].shift(1) & 
                                             df1['C_LongBody'].shift(1))     &  ( df1['C_BlackBody'] &
                                             ( df1['open'] >=  df1['high'].shift(1)) & (df1['close']< df1['C_BodyMiddle'].shift(1)) & 
                                                        (df1['close'] > df1['open'].shift(1)) ) ), "1", "0" )
    
    return df1


In [22]:
## function for grouping by frequency 
def FrameChanging(df, x):
    t = df.groupby(pd.Grouper(freq=x)).agg({"open": "first", 
                                             "close": "last", 
                                             "low": "min", 
                                             "high": "max",
                                             "volume": "sum"})
    return t

In [23]:
##adding column dateindex of data type datetime to set as index of data frame
def DatePre(df):
    df['DateIndex'] = pd.to_datetime(df['timestamp'])
    df = df.set_index('DateIndex')
    df = df.drop(['timestamp'], axis=1)
    return df

In [24]:
##reading sample dataset of adani power for identifying above pattern
df = pd.read_csv("../Data/ADANIPOWER__EQ__NSE__NSE__MINUTE.csv")

In [25]:
df['DateIndex'] = pd.to_datetime(df['timestamp'])
df = df.set_index('DateIndex')
df = df.drop(['timestamp'], axis=1)

In [26]:
#grouping by 30 mins frequency 
df1 = FrameChanging(df, '30Min')

In [27]:
#dealing with missing values
df1 = df1.dropna()

functions defined above result in dataframes with added columns relevant and required for pattern analysis, each cell is marked with the trend the datapoint falls under 

In [104]:
df1M = AbandonedBabyBear(df1)
df1M.tail(20)

,open,high,low,close,volume,MA100,MA200,MA50,C_DownTrend,C_UpTrend,...,C_HasUpShadow,C_HasDnShadow,C_WhiteBody,C_BlackBody,C_Range,C_BodyMiddle,C_ShadowEquals,C_IsDojiBody,C_Doji,C_AbandonedBabyBearish
DateIndex,,,,,,,,,,,,,,,,,,,,,
2020-06-26 15:10:00+05:30,36.95,36.95,36.90,36.90,51193.0,37.0330,37.16650,36.920,True,False,...,0,0,False,True,0.05,0.001285,False,False,False,0
2020-06-26 15:11:00+05:30,36.90,36.95,36.85,36.90,32214.0,37.0300,37.16400,36.919,True,False,...,0,0,False,False,0.10,0.000000,True,True,True,0
2020-06-26 15:12:00+05:30,36.90,36.95,36.85,36.85,48603.0,37.0260,37.16175,36.916,True,False,...,0,0,False,True,0.10,0.001287,True,False,False,0
2020-06-26 15:13:00+05:30,36.85,36.95,36.85,36.95,11593.0,37.0230,37.16025,36.916,False,False,...,0,0,True,False,0.10,0.002574,False,False,False,0
2020-06-26 15:14:00+05:30,36.95,36.95,36.85,36.85,47337.0,37.0190,37.15825,36.913,True,False,...,0,0,False,True,0.10,0.002574,False,False,False,0
2020-06-26 15:15:00+05:30,36.85,36.90,36.85,36.85,12916.0,37.0155,37.15625,36.911,True,False,...,0,0,False,False,0.05,0.000000,True,True,True,0
2020-06-26 15:16:00+05:30,36.90,36.90,36.80,36.80,46898.0,37.0110,37.15350,36.908,True,False,...,0,0,False,True,0.10,0.002577,False,False,False,0
2020-06-26 15:17:00+05:30,36.80,36.90,36.80,36.85,18162.0,37.0075,37.15125,36.907,True,False,...,0,0,True,False,0.10,0.001289,True,False,False,0
2020-06-26 15:18:00+05:30,36.90,36.90,36.45,36.45,356224.0,36.9995,37.14675,36.898,True,False,...,0,0,False,True,0.45,0.011704,False,False,False,0


In [29]:
df1HM = HangingManBear(df1)

In [30]:
df1DJSBear = DojiStarBear(df)

In [31]:
df1DJSBear['C_DojiStarBearish'].value_counts()

0    320975
1       386
Name: C_DojiStarBearish, dtype: int64

In [32]:
df1HM['C_HangingManBearish'].value_counts()

0    9473
1    1667
Name: C_HangingManBearish, dtype: int64

In [33]:
df1DJSBull = DojiStarBull(df1)

In [34]:
df1DJSBull['C_DojiStarBullish'].value_counts()

0    11130
1       10
Name: C_DojiStarBullish, dtype: int64

In [35]:
df1DCC = DarkCloudCover(df1)

In [36]:
df1DCC['C_DarkCloudCoverBearish'].value_counts()

0    11140
Name: C_DarkCloudCoverBearish, dtype: int64

In [37]:
df1ABBull = AbandonedBabyBull(df1)

In [38]:
df1ABBull['C_AbandonedBabyBullish'].value_counts()

0    321361
Name: C_AbandonedBabyBullish, dtype: int64

In [39]:
def Doji(df):
    df1 = df.copy()
    #moving average calculated over 100 200 and 50 days
    df1['MA100'] = df.rolling(window=100,min_periods=1 ).mean()['close']
    df1['MA200'] = df.rolling(window=200,min_periods=1 ).mean()['close']
    df1['MA50'] = df.rolling(window=50,min_periods=1 ).mean()['close']
    C_DownTrend = True
    C_UpTrend = True
    #marking C_DownTrend and C_UpTrend as true or false according to respective conditions and patterns
    df1['C_DownTrend'] = np.where((  (df1['close'] < df1['MA50']) & (df1['MA50'] < df1['MA200']) ), True, False)
    df1['C_UpTrend'] = np.where(( (df1['close'] > df1['MA50'])& (df1['MA50'] > df1['MA200']) ), True, False)
    C_Len = 14 #ema depth for bodyAvg
    C_ShadowPercent = 5.0 # size of shadows
    C_ShadowEqualsPercent = 100.0
    C_DojiBodyPercent = 5.0
    C_Factor = 2.0 # shows the number of times the shadow dominates the candlestick body

    #finding body high and low and body length from closing and opening price of the day
    df1['C_BodyHi'] = df1[['close','open']].max(axis=1)
    df1['C_BodyLo'] = df1[['close','open']].min(axis=1)
    df1['C_Body'] = df1['C_BodyHi'] - df1['C_BodyLo']
    
    df1['C_BodyAvg'] = pd.Series.ewm(df1['C_Body'], span=C_Len).mean()
    
    #setting C_SmallBody as 1 if body average is > the body length and 0 if body average is <= to body length
    df1.loc[df1['C_BodyAvg']> df1['C_Body']  , 'C_SmallBody'] = True
    df1.loc[df1['C_BodyAvg']<= df1['C_Body']  , 'C_SmallBody'] = False

    #setting C_LongBody as 1 if body average is < the body length and 0 if body average is >= to body length
    df1.loc[df1['C_BodyAvg']< df1['C_Body']  , 'C_LongBody'] = True
    df1.loc[df1['C_BodyAvg']>= df1['C_Body']  , 'C_LongBody'] = False
    
    #calculating upshadow as high - body high and down shadow as body low - low
    
    df1['C_UpShadow'] = df1['high'] - df1['C_BodyHi']

    df1['C_DnShadow'] = df1['C_BodyLo'] - df1['low']
    #setting C_HasUpShadow as 0 or 1 indicating presence of up shadow
    df1['C_HasUpShadow'] = np.where((df1.C_UpShadow  > (C_ShadowPercent / (100*df1.C_Body))  ),
                                                     True, False)
    #setting C_HasDnShadow as 0 or 1 indicating presence of down shadow
    df1['C_HasDnShadow'] = np.where((df1.C_DnShadow  > (C_ShadowPercent / (100*df1.C_Body))  ),
                                                     True, False)
    
     #white body reprecents traditional green candlestick and black represents red
    df1['C_WhiteBody'] = np.where((df1.open  < df1.close ),True, False)
    df1['C_BlackBody'] = np.where((df1.open  > df1.close ),True, False)
    
    df1['C_Range'] = df1['high']-df1['low']
    df1['C_BodyMiddle']= df1['C_Body'] / (2 + df1['C_BodyLo'])
    
    #findng if upshadow is equal to dowm shadow
    df1['C_ShadowEquals'] = np.where(  ((df1['C_UpShadow'] == df1['C_DnShadow']) | 
                                 (( abs(df1['C_UpShadow'] - df1['C_DnShadow']) / (df1['C_DnShadow'] * 100) ) )  < 
                                 C_ShadowEqualsPercent) & (((abs(df1['C_DnShadow'] - df1['C_UpShadow']) / (df1['C_UpShadow'] * 100))
                                 < C_ShadowEqualsPercent))  , True, False)
    
    #checking if body range is <= 20 percent of body range of candlestick and setting doji body as true or false accordingly
    df1['C_IsDojiBody'] = np.where(( (df1['C_Range'] > 0)  & (df1['C_Body'] <= (df1['C_Range'] * (C_DojiBodyPercent / 100))) ), True, False)
    
    df1['C_Doji'] = df1['C_IsDojiBody'] & df1['C_ShadowEquals']
    
    df1['hl2'] = (df['high'] + df1['low'])/2
    #A Dragonfly Doji is a type of candlestick pattern that can signal a potential reversal in price to the downside or upside, depending on past price action. It's formed when the asset's high, open, and close prices are the same.
    #gravestone doji is opposite to dragonfly doji
    #setting dragonflydoji to true if it is dojibody and upshadow less than c body
    df1['C_DragonflyDoji'] = np.where( (  df1['C_IsDojiBody'] & (df1['C_UpShadow'] <= df1['C_Body'] )  )   , True, False)
    
     #setting gravestoneydoji to true if it is dojibody and downshadow less than c body
    df1['C_GravestoneDojiOne'] = np.where( (  df1['C_IsDojiBody'] & (df1['C_DnShadow'] <= df1['C_Body'] )  )   , True, False)
    #setting dojires to true if c_doji is true and c_dragonflydoji and c_gravestonedoji is false
    df1['C_DojiRes'] = np.where( ( df1['C_Doji'] & (~ df1['C_DragonflyDoji']) & (~ df1['C_GravestoneDojiOne']) ), "1", "0" )
    
    return df1


In [40]:
df1Doji = Doji(df1)

In [41]:
df1Doji['C_DojiRes'].value_counts()

0    9981
1    1159
Name: C_DojiRes, dtype: int64

The one day Bullish Reversal pattern Dragonfly Doji is a rare candlestick pattern that occurs at the bottom of a downtrend. It is very similar to the Bullish Hammer Pattern, except on a Dragonfly Doji the opening and closing prices are nearly identical with no body.

The Bullish Dragonfly Doji is considered to be more reliable than a Bullish Hammer and tends to be a stronger bullish signal. The pattern is considered most reliable after an established bearish trend. 
 

In [42]:
def DragonFlyDojiBull(df):
    df1 = df.copy()
    

    #moving average calculated over 100 200 and 50 days
    df1['MA100'] = df.rolling(window=100,min_periods=1 ).mean()['close']
    df1['MA200'] = df.rolling(window=200,min_periods=1 ).mean()['close']
    df1['MA50'] = df.rolling(window=50,min_periods=1 ).mean()['close']
    C_DownTrend = True
    C_UpTrend = True
    #marking C_DownTrend and C_UpTrend as true or false according to respective conditions and patterns
    df1['C_DownTrend'] = np.where((  (df1['close'] < df1['MA50']) & (df1['MA50'] < df1['MA200']) ), True, False)
    df1['C_UpTrend'] = np.where(( (df1['close'] > df1['MA50'])& (df1['MA50'] > df1['MA200']) ), True, False)
    C_Len = 14 #ema depth for bodyAvg
    C_ShadowPercent = 5.0 # size of shadows
    C_ShadowEqualsPercent = 100.0
    C_DojiBodyPercent = 5.0
    C_Factor = 2.0 # shows the number of times the shadow dominates the candlestick body
      #finding body high and low and body length from closing and opening price of the day
    df1['C_BodyHi'] = df1[['close','open']].max(axis=1)
    df1['C_BodyLo'] = df1[['close','open']].min(axis=1)
    df1['C_Body'] = df1['C_BodyHi'] - df1['C_BodyLo']
    
    df1['C_BodyAvg'] = pd.Series.ewm(df1['C_Body'], span=C_Len).mean()
    
    #setting C_SmallBody as 1 if body average is > the body length and 0 if body average is <= to body length
    df1.loc[df1['C_BodyAvg']> df1['C_Body']  , 'C_SmallBody'] = True
    df1.loc[df1['C_BodyAvg']<= df1['C_Body']  , 'C_SmallBody'] = False

    #setting C_LongBody as 1 if body average is < the body length and 0 if body average is >= to body length
    df1.loc[df1['C_BodyAvg']< df1['C_Body']  , 'C_LongBody'] = True
    df1.loc[df1['C_BodyAvg']>= df1['C_Body']  , 'C_LongBody'] = False
    
    #calculating upshadow as high - body high and down shadow as body low - low
    df1['C_UpShadow'] = df1['high'] - df1['C_BodyHi']
    df1['C_DnShadow'] = df1['C_BodyLo'] - df1['low']

     #setting C_HasUpShadow as 0 or 1 indicating presence of up shadow
    df1['C_HasUpShadow'] = np.where((df1.C_UpShadow  > (C_ShadowPercent / (100*df1.C_Body))  ),
                                                     True, False)

     #setting C_HasDnShadow as 0 or 1 indicating presence of down shadow                                                
    df1['C_HasDnShadow'] = np.where((df1.C_DnShadow  > (C_ShadowPercent / (100*df1.C_Body))  ),
                                                     True, False)
    

    #white body reprecents traditional green candlestick and black represents red
    df1['C_WhiteBody'] = np.where((df1.open  < df1.close ),True, False)
    df1['C_BlackBody'] = np.where((df1.open  > df1.close ),True, False)
    
    df1['C_Range'] = df1['high']-df1['low']
    df1['C_BodyMiddle']= df1['C_Body'] / (2 + df1['C_BodyLo'])
    
    #findng if upshadow is equal to dowm shadow
    df1['C_ShadowEquals'] = np.where(  ((df1['C_UpShadow'] == df1['C_DnShadow']) | 
                                 (( abs(df1['C_UpShadow'] - df1['C_DnShadow']) / (df1['C_DnShadow'] * 100) ) )  < 
                                 C_ShadowEqualsPercent) & (((abs(df1['C_DnShadow'] - df1['C_UpShadow']) / (df1['C_UpShadow'] * 100))
                                 < C_ShadowEqualsPercent))  , True, False)
    #checking if body range is <= 20 percent of body range of candlestick and setting doji body as true or false accordingly
    df1['C_IsDojiBody'] = np.where(( (df1['C_Range'] > 0)  & (df1['C_Body'] <= (df1['C_Range'] * (C_DojiBodyPercent / 100))) ), True, False)
    
    #c_doji set to true if both isdojibody and shadowequals are true else false
    df1['C_Doji'] = df1['C_IsDojiBody'] & df1['C_ShadowEquals']
    
    df1['hl2'] = (df['high'] + df1['low'])/2
     #setting dragonflydoji to true if it is dojibody and upshadow less than c body
    df1['C_DragonflyDoji'] = np.where( (  df1['C_IsDojiBody'] & (df1['C_UpShadow'] <= df1['C_Body'] )  )   , True, False)
    

    return df1


In [43]:
df1DragonDoji = Doji(df1)

In [44]:
df1DragonDoji['C_DragonflyDoji'].value_counts()

False    11068
True        72
Name: C_DragonflyDoji, dtype: int64

A bearish engulfing pattern is a technical chart pattern that signals lower prices to come. The pattern consists of an up (white or green) candlestick followed by a large down (black or red) candlestick that eclipses or "engulfs" the smaller up candle. The pattern can be important because it shows sellers have overtaken the buyers and are pushing the price more aggressively down (down candle) than the buyers were able to push it up (up candle).

In [45]:
def EngulfingBear(df):
    df1 = df.copy()
    df1 = df.copy()
    #moving average calculated over 100 200 and 50 days
    df1['MA100'] = df.rolling(window=100,min_periods=1 ).mean()['close']
    df1['MA200'] = df.rolling(window=200,min_periods=1 ).mean()['close']
    df1['MA50'] = df.rolling(window=50,min_periods=1 ).mean()['close']
    C_DownTrend = True
    C_UpTrend = True
        #marking C_DownTrend and C_UpTrend as true or false according to respective conditions and patterns
    df1['C_DownTrend'] = np.where((  (df1['close'] < df1['MA50']) & (df1['MA50'] < df1['MA200']) ), True, False)
    df1['C_UpTrend'] = np.where(( (df1['close'] > df1['MA50'])& (df1['MA50'] > df1['MA200']) ), True, False)
    C_Len = 14 #ema depth for bodyAvg
    C_ShadowPercent = 5.0 # size of shadows
    C_ShadowEqualsPercent = 100.0
    C_DojiBodyPercent = 5.0
    C_Factor = 2.0 # shows the number of times the shadow dominates the candlestick body

    #finding body high and low and body length from closing and opening price of the day
    df1['C_BodyHi'] = df1[['close','open']].max(axis=1)
    df1['C_BodyLo'] = df1[['close','open']].min(axis=1)
    df1['C_Body'] = df1['C_BodyHi'] - df1['C_BodyLo']
    
    df1['C_BodyAvg'] = pd.Series.ewm(df1['C_Body'], span=C_Len).mean()
    
        #setting C_SmallBody as 1 if body average is > the body length and 0 if body average is <= to body length
    df1.loc[df1['C_BodyAvg']> df1['C_Body']  , 'C_SmallBody'] = True
    df1.loc[df1['C_BodyAvg']<= df1['C_Body']  , 'C_SmallBody'] = False

        #setting C_LongBody as 1 if body average is < the body length and 0 if body average is >= to body length
    df1.loc[df1['C_BodyAvg']< df1['C_Body']  , 'C_LongBody'] = True
    df1.loc[df1['C_BodyAvg']>= df1['C_Body']  , 'C_LongBody'] = False
    
    
   #calculating upshadow as high - body high and down shadow as body low - low
    df1['C_UpShadow'] = df1['high'] - df1['C_BodyHi']
   
    df1['C_DnShadow'] = df1['C_BodyLo'] - df1['low']
       #setting C_HasUpShadow as 0 or 1 indicating presence of up shadow
    df1['C_HasUpShadow'] = np.where((df1.C_UpShadow  > (C_ShadowPercent / (100*df1.C_Body))  ),
                                                     True, False)
     #setting C_HasDnShadow as 0 or 1 indicating presence of down shadow
    df1['C_HasDnShadow'] = np.where((df1.C_DnShadow  > (C_ShadowPercent / (100*df1.C_Body))  ),
                                                     True, False)
     #white body reprecents traditional green candlestick and black represents red
    df1['C_WhiteBody'] = np.where((df1.open  < df1.close ),True, False)
    df1['C_BlackBody'] = np.where((df1.open  > df1.close ),True, False)
    
    df1['C_Range'] = df1['high']-df1['low']
    df1['C_BodyMiddle']= df1['C_Body'] / (2 + df1['C_BodyLo'])
    #findng if upshadow is equal to dowm shadow
    df1['C_ShadowEquals'] = np.where(  ((df1['C_UpShadow'] == df1['C_DnShadow']) | 
                                 (( abs(df1['C_UpShadow'] - df1['C_DnShadow']) / (df1['C_DnShadow'] * 100) ) )  < 
                                 C_ShadowEqualsPercent) & (((abs(df1['C_DnShadow'] - df1['C_UpShadow']) / (df1['C_UpShadow'] * 100))
                                 < C_ShadowEqualsPercent))  , True, False)
    
    #checking if body range is <= 20 percent of body range of candlestick and setting doji body as true or false accordingly
    df1['C_IsDojiBody'] = np.where(( (df1['C_Range'] > 0)  & (df1['C_Body'] <= (df1['C_Range'] * (C_DojiBodyPercent / 100))) ), True, False)
    
    df1['C_Doji'] = df1['C_IsDojiBody'] & df1['C_ShadowEquals']
    
    
    df1['hl2'] = (df['high'] + df1['low'])/2
    
    #checking condition for engulfing bearish and setting to true or false respectively
    df1['C_EngulfingBearish'] = np.where( ( df1['C_UpTrend']  &  df1['C_BlackBody'] &
                                          df1['C_LongBody'] &  df1['C_WhiteBody'].shift(1) &
                                          df1['C_SmallBody'].shift(1) &
                                            (df1['close'] <= df1['open'].shift(1))  &
                                           (df1['open'] >= df1['close'].shift(1)) &
                                          ( (df1['close'] <= df1['open'].shift(1)) | (df1['open'] > df1['close'].shift(1)) ))   , True, False)
    

    return df1


In [46]:
df1EngulBear = EngulfingBear(df1)
print(df1EngulBear['C_EngulfingBearish'].value_counts())

False    11040
True       100
Name: C_EngulfingBearish, dtype: int64


Bullish engulfing pattern:
The bullish engulfing pattern is a two-candle reversal pattern. The second candle completely ‘engulfs’ the real body of the first one, without regard to the length of the tail shadows.
This pattern appears in a downtrend and is a combination of one dark candle followed by a larger hollow candle. On the second day of the pattern, the price opens lower than the previous low, yet buying pressure pushes the price up to a higher level than the previous high, culminating in an obvious win for the buyers.

In [47]:
def EngulfingBull(df):
    df1 = df.copy()
     #moving average calculated over 100 200 and 50 days
    df1['MA100'] = df.rolling(window=100,min_periods=1 ).mean()['close']
    df1['MA200'] = df.rolling(window=200,min_periods=1 ).mean()['close']
    df1['MA50'] = df.rolling(window=50,min_periods=1 ).mean()['close']
    C_DownTrend = True
    C_UpTrend = True
     #marking C_DownTrend and C_UpTrend as true or false according to respective conditions and patterns
    df1['C_DownTrend'] = np.where((  (df1['close'] < df1['MA50']) & (df1['MA50'] < df1['MA200']) ), True, False)
    df1['C_UpTrend'] = np.where(( (df1['close'] > df1['MA50'])& (df1['MA50'] > df1['MA200']) ), True, False)
    C_Len = 14 #ema depth for bodyAvg
    C_ShadowPercent = 5.0 # size of shadows
    C_ShadowEqualsPercent = 100.0
    C_DojiBodyPercent = 5.0
    C_Factor = 2.0 # shows the number of times the shadow dominates the candlestick body
        #finding body high and low and body length from closing and opening price of the day
    df1['C_BodyHi'] = df1[['close','open']].max(axis=1)
    df1['C_BodyLo'] = df1[['close','open']].min(axis=1)
    df1['C_Body'] = df1['C_BodyHi'] - df1['C_BodyLo']
    
    df1['C_BodyAvg'] = pd.Series.ewm(df1['C_Body'], span=C_Len).mean()
        #setting C_SmallBody as 1 if body average is > the body length and 0 if body average is <= to body length
    
    df1.loc[df1['C_BodyAvg']> df1['C_Body']  , 'C_SmallBody'] = True
    df1.loc[df1['C_BodyAvg']<= df1['C_Body']  , 'C_SmallBody'] = False

     #setting C_LongBody as 1 if body average is < the body length and 0 if body average is >= to body length
    df1.loc[df1['C_BodyAvg']< df1['C_Body']  , 'C_LongBody'] = True
    df1.loc[df1['C_BodyAvg']>= df1['C_Body']  , 'C_LongBody'] = False
    
    #calculating upshadow as high - body high and down shadow as body low - low
    df1['C_UpShadow'] = df1['high'] - df1['C_BodyHi']
    df1['C_DnShadow'] = df1['C_BodyLo'] - df1['low']
     #setting C_HasUpShadow as 0 or 1 indicating presence of up shadow
    df1['C_HasUpShadow'] = np.where((df1.C_UpShadow  > (C_ShadowPercent / (100*df1.C_Body))  ),
                                                     True, False)

         #setting C_HasDnShadow as 0 or 1 indicating presence of down shadow                                                
    df1['C_HasDnShadow'] = np.where((df1.C_DnShadow  > (C_ShadowPercent / (100*df1.C_Body))  ),
                                                     True, False)
     #white body reprecents traditional green candlestick and black represents red
    df1['C_WhiteBody'] = np.where((df1.open  < df1.close ),True, False)
    df1['C_BlackBody'] = np.where((df1.open  > df1.close ),True, False)
    
    df1['C_Range'] = df1['high']-df1['low']
    df1['C_BodyMiddle']= df1['C_Body'] / (2 + df1['C_BodyLo'])
    
       
    #findng if upshadow is equal to dowm shadow
    df1['C_ShadowEquals'] = np.where(  ((df1['C_UpShadow'] == df1['C_DnShadow']) | 
                                 (( abs(df1['C_UpShadow'] - df1['C_DnShadow']) / (df1['C_DnShadow'] * 100) ) )  < 
                                 C_ShadowEqualsPercent) & (((abs(df1['C_DnShadow'] - df1['C_UpShadow']) / (df1['C_UpShadow'] * 100))
                                 < C_ShadowEqualsPercent))  , True, False)
    
        #checking if body range is <= 20 percent of body range of candlestick and setting doji body as true or false accordingly
    df1['C_IsDojiBody'] = np.where(( (df1['C_Range'] > 0)  & (df1['C_Body'] <= (df1['C_Range'] * (C_DojiBodyPercent / 100))) ), True, False)
    
    df1['C_Doji'] = df1['C_IsDojiBody'] & df1['C_ShadowEquals']
    
    
    df1['hl2'] = (df['high'] + df1['low'])/2
        #checking condition for engulfing bullish and setting to true or false respectively
    df1['C_EngulfingBullish'] = np.where( ( df1['C_DownTrend']  &  df1['C_WhiteBody'] &
                                           df1['C_LongBody'] &  df1['C_BlackBody'].shift(1) &
                                          df1['C_SmallBody'].shift(1) &
                                            (df1['close'] >= df1['open'].shift(1))  &
                                           (df1['open'] <= df1['close'].shift(1)) &
                                          ( (df1['close'] > df1['open'].shift(1)) | (df1['open'] <df1['close'].shift(1)) ))   , "1", "0")
    

    return df1


In [48]:
df1EngulBull = EngulfingBull(df1)
print(df1EngulBull['C_EngulfingBullish'].value_counts())

0    11028
1      112
Name: C_EngulfingBullish, dtype: int64


Evening Doji star Bear:
An evening star is a stock-price chart pattern used by technical analysts to detect when a trend is about to reverse. It is a bearish candlestick pattern consisting of three candles: a large white candlestick, a small-bodied candle, and a red candle. The Evening Doji Star is a bearish reversal pattern, being very similar to the Evening Star. The only difference is that the Evening Doji Star needs to have a doji candle (except the Four-Price Doji) on the second line. 

Construction:

First candle
a candle in an uptrend
white body
Second candle
a doji candle
a doji body above the previous candle body
the low price below the previous candle high price
Third candle
black body
candle body below the previous candle body
the closing price below the midpoint of the first candle body


In [49]:

def EveningDojiBear(df):
    
    df1 = df.copy()
    
    df1['MA100'] = df.rolling(window=100,min_periods=1 ).mean()['close']
    df1['MA200'] = df.rolling(window=200,min_periods=1 ).mean()['close']
    df1['MA50'] = df.rolling(window=50,min_periods=1 ).mean()['close']
    C_DownTrend = True
    C_UpTrend = True
    df1['C_DownTrend'] = np.where((  (df1['close'] < df1['MA50']) & (df1['MA50'] < df1['MA200']) ), True, False)
    df1['C_UpTrend'] = np.where(( (df1['close'] > df1['MA50'])& (df1['MA50'] > df1['MA200']) ), True, False)
    C_Len = 14 #ema depth for bodyAvg
    C_ShadowPercent = 5.0 # size of shadows
    C_ShadowEqualsPercent = 100.0
    C_DojiBodyPercent = 5.0
    C_Factor = 2.0 # shows the number of times the shadow dominates the candlestick body
    df1['C_BodyHi'] = df1[['close','open']].max(axis=1)
    df1['C_BodyLo'] = df1[['close','open']].min(axis=1)
    df1['C_Body'] = df1['C_BodyHi'] - df1['C_BodyLo']
    
    df1['C_BodyAvg'] = pd.Series.ewm(df1['C_Body'], span=C_Len).mean()
    
    
    df1.loc[df1['C_BodyAvg']> df1['C_Body']  , 'C_SmallBody'] = True
    df1.loc[df1['C_BodyAvg']<= df1['C_Body']  , 'C_SmallBody'] = False

    df1.loc[df1['C_BodyAvg']< df1['C_Body']  , 'C_LongBody'] = True
    df1.loc[df1['C_BodyAvg']>= df1['C_Body']  , 'C_LongBody'] = False
    
    
    df1['C_UpShadow'] = df1['high'] - df1['C_BodyHi']
    df1['C_DnShadow'] = df1['C_BodyLo'] - df1['low']
    df1['C_HasUpShadow'] = np.where((df1.C_UpShadow  > (C_ShadowPercent / (100*df1.C_Body))  ),
                                                     True, False)
    df1['C_HasDnShadow'] = np.where((df1.C_DnShadow  > (C_ShadowPercent / (100*df1.C_Body))  ),
                                                     True, False)
    
    df1['C_WhiteBody'] = np.where((df1.open  < df1.close ),True, False)
    df1['C_BlackBody'] = np.where((df1.open  > df1.close ),True, False)
    
    df1['C_Range'] = df1['high']-df1['low']
    df1['C_BodyMiddle']= df1['C_Body'] / (2 + df1['C_BodyLo'])
    
    df1['C_ShadowEquals'] = np.where(  ((df1['C_UpShadow'] == df1['C_DnShadow']) | 
                                 (( abs(df1['C_UpShadow'] - df1['C_DnShadow']) / (df1['C_DnShadow'] * 100) ) )  < 
                                 C_ShadowEqualsPercent) & (((abs(df1['C_DnShadow'] - df1['C_UpShadow']) / (df1['C_UpShadow'] * 100))
                                 < C_ShadowEqualsPercent))  , True, False)
    
    df1['C_IsDojiBody'] = np.where(( (df1['C_Range'] > 0)  & (df1['C_Body'] <= (df1['C_Range'] * (C_DojiBodyPercent / 100))) ), True, False)
    
    df1['C_Doji'] = df1['C_IsDojiBody'] & df1['C_ShadowEquals']
    
    df1['hl2'] = (df['high'] + df1['low'])/2

    #checking above construction conditions to see if datapoint forms part of evening doji star 
    df1['C_EveningDojiStarBearish'] = np.where(( df1['C_LongBody'].shift(2) & df1['C_IsDojiBody'].shift(1) &
                                            df1['C_LongBody'] & df1['C_UpTrend'] &  df1['C_WhiteBody'] &
                                             (df1['C_BodyLo'].shift(1) > df1['C_BodyHi'].shift(2)) &
                                              df1['C_BlackBody'] &  (df1['C_BodyLo'] <= df1['C_BodyMiddle'].shift(2)) &
                                              (df1['C_BodyLo'] > df1['C_BodyLo'].shift(2)) & 
                                               (df1['C_BodyLo'].shift(2) > df1['C_BodyHi'])), "1", "0" )
    
    return df1

In [50]:
df1EveningDoji = EveningDojiBear(df1)
print(df1EveningDoji['C_EveningDojiStarBearish'].value_counts())

0    11140
Name: C_EveningDojiStarBearish, dtype: int64


Evening Star bearish Pattern:

Evening star patterns are associated with the top of a price uptrend, signifying that the uptrend is nearing its end.

The evening star pattern is considered a very strong indicator of future price declines. Its pattern forms over a period of three days:

The first day consists of a large white candle signifying a continued rise in prices.
The second day consists of a smaller candle that shows a more modest increase in price.
The third day shows a large red candle that opens at a price below the previous day and then closes near the middle of the first day.

In [51]:
def EveningStarBear(df):
    
    df1 = df.copy()
    
    df1['MA100'] = df.rolling(window=100,min_periods=1 ).mean()['close']
    df1['MA200'] = df.rolling(window=200,min_periods=1 ).mean()['close']
    df1['MA50'] = df.rolling(window=50,min_periods=1 ).mean()['close']
    C_DownTrend = True
    C_UpTrend = True
    df1['C_DownTrend'] = np.where((  (df1['close'] < df1['MA50']) & (df1['MA50'] < df1['MA200']) ), True, False)
    df1['C_UpTrend'] = np.where(( (df1['close'] > df1['MA50'])& (df1['MA50'] > df1['MA200']) ), True, False)
    C_Len = 14 #ema depth for bodyAvg
    C_ShadowPercent = 5.0 # size of shadows
    C_ShadowEqualsPercent = 100.0
    C_DojiBodyPercent = 5.0
    C_Factor = 2.0 # shows the number of times the shadow dominates the candlestick body
    df1['C_BodyHi'] = df1[['close','open']].max(axis=1)
    df1['C_BodyLo'] = df1[['close','open']].min(axis=1)
    df1['C_Body'] = df1['C_BodyHi'] - df1['C_BodyLo']
    
    df1['C_BodyAvg'] = pd.Series.ewm(df1['C_Body'], span=C_Len).mean()
    
    
    df1.loc[df1['C_BodyAvg']> df1['C_Body']  , 'C_SmallBody'] = True
    df1.loc[df1['C_BodyAvg']<= df1['C_Body']  , 'C_SmallBody'] = False

    df1.loc[df1['C_BodyAvg']< df1['C_Body']  , 'C_LongBody'] = True
    df1.loc[df1['C_BodyAvg']>= df1['C_Body']  , 'C_LongBody'] = False
    
    
    df1['C_UpShadow'] = df1['high'] - df1['C_BodyHi']
    df1['C_DnShadow'] = df1['C_BodyLo'] - df1['low']
    df1['C_HasUpShadow'] = np.where((df1.C_UpShadow  > (C_ShadowPercent / (100*df1.C_Body))  ),
                                                     True, False)
    df1['C_HasDnShadow'] = np.where((df1.C_DnShadow  > (C_ShadowPercent / (100*df1.C_Body))  ),
                                                     True, False)
    
    df1['C_WhiteBody'] = np.where((df1.open  < df1.close ),True, False)
    df1['C_BlackBody'] = np.where((df1.open  > df1.close ),True, False)
    
    df1['C_Range'] = df1['high']-df1['low']
    df1['C_BodyMiddle']= df1['C_Body'] / (2 + df1['C_BodyLo'])
    
    df1['C_ShadowEquals'] = np.where(  ((df1['C_UpShadow'] == df1['C_DnShadow']) | 
                                 (( abs(df1['C_UpShadow'] - df1['C_DnShadow']) / (df1['C_DnShadow'] * 100) ) )  < 
                                 C_ShadowEqualsPercent) & (((abs(df1['C_DnShadow'] - df1['C_UpShadow']) / (df1['C_UpShadow'] * 100))
                                 < C_ShadowEqualsPercent))  , True, False)
    
    df1['C_IsDojiBody'] = np.where(( (df1['C_Range'] > 0)  & (df1['C_Body'] <= (df1['C_Range'] * (C_DojiBodyPercent / 100))) ), True, False)
    
    df1['C_Doji'] = df1['C_IsDojiBody'] & df1['C_ShadowEquals']
    
    df1['hl2'] = (df['high'] + df1['low'])/2


      #checking above construction conditions to see if datapoint forms part of evening  star 
    df1['C_EveningStarBearish'] = np.where((df1['C_LongBody'].shift(2) &
                                            df1['C_SmallBody'].shift(1) & 
                                            df1['C_LongBody'] &
                                           df1['C_UpTrend'] & df1['C_WhiteBody'].shift(2) &
                                           (df1['C_BodyLo'].shift(1) > df1['C_BodyHi'].shift(2)) &
                                           df1['C_BlackBody'] & 
                                            (df1['C_BodyLo'] <= df1['C_BodyMiddle'].shift(2)) &
                                           (df1['C_BodyLo'] > df1['C_BodyLo'].shift(2)) &
                                           (df1['C_BodyLo'].shift(1) > df1['C_BodyHi'])), "1", "0" )
    
    return df1

In [52]:
df1EveningStar = EveningStarBear(df1)
print(df1EveningStar['C_EveningStarBearish'].value_counts())

0    11140
Name: C_EveningStarBearish, dtype: int64


falling window bearish pattern:
The falling window  occurs when yesterday's low is above today's high, leaving a hole on the daily price chart. The pattern appears in a falling price trend, and it acts as a bearish continuation pattern.

Construction:

First candle
any candle except the Four-Price Doji
a candle in a downtrend or closing below the trendline
Second candle
any candle except the Four-Price Doji
the candle's high below the previous candle's low


In [53]:
def FallingWindowBear(df):
    
    df1 = df.copy()
    
    df1['MA100'] = df.rolling(window=100,min_periods=1 ).mean()['close']
    df1['MA200'] = df.rolling(window=200,min_periods=1 ).mean()['close']
    df1['MA50'] = df.rolling(window=50,min_periods=1 ).mean()['close']
    C_DownTrend = True
    C_UpTrend = True
    df1['C_DownTrend'] = np.where((  (df1['close'] < df1['MA50']) & (df1['MA50'] < df1['MA200']) ), True, False)
    df1['C_UpTrend'] = np.where(( (df1['close'] > df1['MA50'])& (df1['MA50'] > df1['MA200']) ), True, False)
    C_Len = 14 #ema depth for bodyAvg
    C_ShadowPercent = 5.0 # size of shadows
    C_ShadowEqualsPercent = 100.0
    C_DojiBodyPercent = 5.0
    C_Factor = 2.0 # shows the number of times the shadow dominates the candlestick body
    df1['C_BodyHi'] = df1[['close','open']].max(axis=1)
    df1['C_BodyLo'] = df1[['close','open']].min(axis=1)
    df1['C_Body'] = df1['C_BodyHi'] - df1['C_BodyLo']
    
    df1['C_BodyAvg'] = pd.Series.ewm(df1['C_Body'], span=C_Len).mean()
    
    
    df1.loc[df1['C_BodyAvg']> df1['C_Body']  , 'C_SmallBody'] = True
    df1.loc[df1['C_BodyAvg']<= df1['C_Body']  , 'C_SmallBody'] = False

    df1.loc[df1['C_BodyAvg']< df1['C_Body']  , 'C_LongBody'] = True
    df1.loc[df1['C_BodyAvg']>= df1['C_Body']  , 'C_LongBody'] = False
    
    
    df1['C_UpShadow'] = df1['high'] - df1['C_BodyHi']
    df1['C_DnShadow'] = df1['C_BodyLo'] - df1['low']
    df1['C_HasUpShadow'] = np.where((df1.C_UpShadow  > (C_ShadowPercent / (100*df1.C_Body))  ),
                                                     True, False)
    df1['C_HasDnShadow'] = np.where((df1.C_DnShadow  > (C_ShadowPercent / (100*df1.C_Body))  ),
                                                     True, False)
    
    df1['C_WhiteBody'] = np.where((df1.open  < df1.close ),True, False)
    df1['C_BlackBody'] = np.where((df1.open  > df1.close ),True, False)
    
    df1['C_Range'] = df1['high']-df1['low']
    df1['C_BodyMiddle']= df1['C_Body'] / (2 + df1['C_BodyLo'])
    
    df1['C_ShadowEquals'] = np.where(  ((df1['C_UpShadow'] == df1['C_DnShadow']) | 
                                 (( abs(df1['C_UpShadow'] - df1['C_DnShadow']) / (df1['C_DnShadow'] * 100) ) )  < 
                                 C_ShadowEqualsPercent) & (((abs(df1['C_DnShadow'] - df1['C_UpShadow']) / (df1['C_UpShadow'] * 100))
                                 < C_ShadowEqualsPercent))  , True, False)
    
    df1['C_IsDojiBody'] = np.where(( (df1['C_Range'] > 0)  & (df1['C_Body'] <= (df1['C_Range'] * (C_DojiBodyPercent / 100))) ), True, False)
    
    df1['C_Doji'] = df1['C_IsDojiBody'] & df1['C_ShadowEquals']
    
    df1['hl2'] = (df['high'] + df1['low'])/2
    
      #checking above construction conditions to see if datapoint forms part of falling window 
    df1['C_FallingWindowBearish'] = np.where((df1['C_DownTrend'].shift(1) &
                                               ( (df1['C_Range'] != 0) & (df1['C_Range'].shift(1) != 0) ) &
                                               (df1['high'] < df1['low'].shift(1)) ), "1", "0" )
    
    return df1

In [54]:
df1FallingWindBear = FallingWindowBear(df1)
print(df1FallingWindBear['C_FallingWindowBearish'].value_counts())

0    11123
1       17
Name: C_FallingWindowBearish, dtype: int64


. A gravestone doji is a bearish pattern that suggests a reversal followed by a downtrend in the price action. A gravestone pattern can be used as a sign to take profits on a bullish position or enter a bearish trade. 

A gravestone doji pattern implies that a bearish reversal is coming. While the open, low, and closing prices don't have to be equal for the pattern to be valid, there should be a relatively small tail, else the pattern could be classified as an inverted hammer, shooting star, or a spinning top. The market narrative is that the bulls attempt to push to new highs over the session, but the bears push the price action to near the open by the session close. So the long upper shadow represents the bulls losing momentum.

While the gravestone doji can be found at the end of a downtrend, it's more common to be found at the end of an uptrend.

In [55]:
def GravestoneDojiBear(df):
    df1 = df.copy()
    
    df1['MA100'] = df.rolling(window=100,min_periods=1 ).mean()['close']
    df1['MA200'] = df.rolling(window=200,min_periods=1 ).mean()['close']
    df1['MA50'] = df.rolling(window=50,min_periods=1 ).mean()['close']
    C_DownTrend = True
    C_UpTrend = True
    df1['C_DownTrend'] = np.where((  (df1['close'] < df1['MA50']) & (df1['MA50'] < df1['MA200']) ), True, False)
    df1['C_UpTrend'] = np.where(( (df1['close'] > df1['MA50'])& (df1['MA50'] > df1['MA200']) ), True, False)
    C_Len = 14 #ema depth for bodyAvg
    C_ShadowPercent = 5.0 # size of shadows
    C_ShadowEqualsPercent = 100.0
    C_DojiBodyPercent = 5.0
    C_Factor = 2.0 # shows the number of times the shadow dominates the candlestick body
    df1['C_BodyHi'] = df1[['close','open']].max(axis=1)
    df1['C_BodyLo'] = df1[['close','open']].min(axis=1)
    df1['C_Body'] = df1['C_BodyHi'] - df1['C_BodyLo']
    
    df1['C_BodyAvg'] = pd.Series.ewm(df1['C_Body'], span=C_Len).mean()
    
    
    df1.loc[df1['C_BodyAvg']> df1['C_Body']  , 'C_SmallBody'] = True
    df1.loc[df1['C_BodyAvg']<= df1['C_Body']  , 'C_SmallBody'] = False

    df1.loc[df1['C_BodyAvg']< df1['C_Body']  , 'C_LongBody'] = True
    df1.loc[df1['C_BodyAvg']>= df1['C_Body']  , 'C_LongBody'] = False
    
    
    df1['C_UpShadow'] = df1['high'] - df1['C_BodyHi']
    df1['C_DnShadow'] = df1['C_BodyLo'] - df1['low']
    df1['C_HasUpShadow'] = np.where((df1.C_UpShadow  > (C_ShadowPercent / (100*df1.C_Body))  ),
                                                     True, False)
    df1['C_HasDnShadow'] = np.where((df1.C_DnShadow  > (C_ShadowPercent / (100*df1.C_Body))  ),
                                                     True, False)
    
    df1['C_WhiteBody'] = np.where((df1.open  < df1.close ),True, False)
    df1['C_BlackBody'] = np.where((df1.open  > df1.close ),True, False)
    
    df1['C_Range'] = df1['high']-df1['low']
    df1['C_BodyMiddle']= df1['C_Body'] / (2 + df1['C_BodyLo'])
    
    df1['C_ShadowEquals'] = np.where(  ((df1['C_UpShadow'] == df1['C_DnShadow']) | 
                                 (( abs(df1['C_UpShadow'] - df1['C_DnShadow']) / (df1['C_DnShadow'] * 100) ) )  < 
                                 C_ShadowEqualsPercent) & (((abs(df1['C_DnShadow'] - df1['C_UpShadow']) / (df1['C_UpShadow'] * 100))
                                 < C_ShadowEqualsPercent))  , True, False)
    
    df1['C_IsDojiBody'] = np.where(( (df1['C_Range'] > 0)  & (df1['C_Body'] <= (df1['C_Range'] * (C_DojiBodyPercent / 100))) ), True, False)
    
    df1['C_Doji'] = df1['C_IsDojiBody'] & df1['C_ShadowEquals']
    
    df1['hl2'] = (df['high'] + df1['low'])/2
   #mark those datapoints to true who have dojibody and downshow < cbody 
    df1['C_GravestoneDojiBearish'] = np.where( (  df1['C_IsDojiBody'] & (df1['C_DnShadow'] <= df1['C_Body'] )  )   , "1", "0")
  
    return df1


In [56]:
df1GraveDojiBear = GravestoneDojiBear(df1)
print(df1GraveDojiBear['C_GravestoneDojiBearish'].value_counts())

0    11045
1       95
Name: C_GravestoneDojiBearish, dtype: int64


The hammer candlestick is found at the bottom of a downtrend and signals a potential (bullish) reversal in the market. A hammer is a candlestick pattern, when a stock opens then moves a lot lower during the day then rallies back near the opening price. The candlestick pattern looks like a hammer with the long lower wick from the lows of the day looking like the handle and the opening and closing price body form what looks like the hammer’s head. The lower wick is usually twice the size of the candle body but can be even bigger.

In [57]:
def HammerBull(df):
    df1 = df.copy()
    
    df1['MA100'] = df1.rolling(window=100,min_periods=1 ).mean()['close']
    df1['MA200'] = df1.rolling(window=200,min_periods=1 ).mean()['close']
    df1['MA50'] = df1.rolling(window=50,min_periods=1 ).mean()['close']
    C_DownTrend = True
    C_UpTrend = True
    df1['C_DownTrend'] = np.where((  (df1['close'] < df1['MA50']) & (df1['MA50'] < df1['MA200']) ), True, False)
    df1['C_UpTrend'] = np.where(( (df1['close'] > df1['MA50'])& (df1['MA50'] > df1['MA200']) ), True, False)
    C_Len = 14 #ema depth for bodyAvg
    C_ShadowPercent = 5.0 # size of shadows
    C_ShadowEqualsPercent = 100.0
    C_DojiBodyPercent = 5.0
    C_Factor = 2.0 # shows the number of times the shadow dominates the candlestick body
    df1['C_BodyHi'] = df1[['close','open']].max(axis=1)
    df1['C_BodyLo'] = df1[['close','open']].min(axis=1)
    df1['C_Body'] = df1['C_BodyHi'] - df1['C_BodyLo']
    
    df1['C_BodyAvg'] = pd.Series.ewm(df1['C_Body'], span=C_Len).mean()
    
    
    df1.loc[df1['C_BodyAvg']> df1['C_Body']  , 'C_SmallBody'] = True
    df1.loc[df1['C_BodyAvg']<= df1['C_Body']  , 'C_SmallBody'] = False

    df1.loc[df1['C_BodyAvg']< df1['C_Body']  , 'C_LongBody'] = True
    df1.loc[df1['C_BodyAvg']>= df1['C_Body']  , 'C_LongBody'] = False
    
    
    df1['C_UpShadow'] = df1['high'] - df1['C_BodyHi']
    df1['C_DnShadow'] = df1['C_BodyLo'] - df1['low']
    df1['C_HasUpShadow'] = np.where((df1.C_UpShadow  > (C_ShadowPercent / (100*df1.C_Body))  ),
                                                     True, False)
    df1['C_HasDnShadow'] = np.where((df1.C_DnShadow  > (C_ShadowPercent / (100*df1.C_Body))  ),
                                                     True, False)
    
    df1['C_WhiteBody'] = np.where((df1.open  < df1.close ),True, False)
    df1['C_BlackBody'] = np.where((df1.open  > df1.close ),True, False)
    
    df1['C_Range'] = df1['high']-df1['low']
    df1['C_BodyMiddle']= df1['C_Body'] / (2 + df1['C_BodyLo'])
    
    df1['C_ShadowEquals'] = np.where(  ((df1['C_UpShadow'] == df1['C_DnShadow']) | 
                                 (( abs(df1['C_UpShadow'] - df1['C_DnShadow']) / (df1['C_DnShadow'] * 100) ) )  < 
                                 C_ShadowEqualsPercent) & (((abs(df1['C_DnShadow'] - df1['C_UpShadow']) / (df1['C_UpShadow'] * 100))
                                 < C_ShadowEqualsPercent))  , True, False)
    
    df1['C_IsDojiBody'] = np.where(( (df1['C_Range'] > 0)  & (df1['C_Body'] <= (df1['C_Range'] * (C_DojiBodyPercent / 100))) ), True, False)
    
    df1['C_Doji'] = df1['C_IsDojiBody'] & df1['C_ShadowEquals']
    
    df1['hl2'] = (df1['high'] + df1['low'])/2
   #marking presence and absence of points that constitute hammer bullish by checking required conditions
    df1['C_HammerBullish'] = np.where( (  df1['C_SmallBody'] & (df1['C_Body'] > 0) &
                                       (df1['C_BodyLo'] > df1['hl2'] ) &
                                      (df1['C_DnShadow'] >= C_Factor * df1['C_Body']) &
                                       (~ df1['C_HasUpShadow']) &
                                       df1['C_DownTrend'] )   , "1", "0")
  
    return df1


In [58]:
df1HammerBull = HammerBull(df1)
print(df1HammerBull['C_HammerBullish'].value_counts())

0    10853
1      287
Name: C_HammerBullish, dtype: int64


A Bearish Harami candlestick is formed when there is a large bullish candle on Day 1and is followed by a smaller bearish candle on Day 2.

One should note that the important aspect of the bearish Harami candlestick is that prices gapped down on Day 2 and also they were unable to move higher back to the close of Day 1.

In [59]:
def HaramiBear(df):
    df1 = df.copy()
    
    df1['MA100'] = df1.rolling(window=100,min_periods=1 ).mean()['close']
    df1['MA200'] = df1.rolling(window=200,min_periods=1 ).mean()['close']
    df1['MA50'] = df1.rolling(window=50,min_periods=1 ).mean()['close']
    C_DownTrend = True
    C_UpTrend = True
    df1['C_DownTrend'] = np.where((  (df1['close'] < df1['MA50']) & (df1['MA50'] < df1['MA200']) ), True, False)
    df1['C_UpTrend'] = np.where(( (df1['close'] > df1['MA50'])& (df1['MA50'] > df1['MA200']) ), True, False)
    C_Len = 14 #ema depth for bodyAvg
    C_ShadowPercent = 5.0 # size of shadows
    C_ShadowEqualsPercent = 100.0
    C_DojiBodyPercent = 5.0
    C_Factor = 2.0 # shows the number of times the shadow dominates the candlestick body
    df1['C_BodyHi'] = df1[['close','open']].max(axis=1)
    df1['C_BodyLo'] = df1[['close','open']].min(axis=1)
    df1['C_Body'] = df1['C_BodyHi'] - df1['C_BodyLo']
    
    df1['C_BodyAvg'] = pd.Series.ewm(df1['C_Body'], span=C_Len).mean()
    
    
    df1.loc[df1['C_BodyAvg']> df1['C_Body']  , 'C_SmallBody'] = True
    df1.loc[df1['C_BodyAvg']<= df1['C_Body']  , 'C_SmallBody'] = False

    df1.loc[df1['C_BodyAvg']< df1['C_Body']  , 'C_LongBody'] = True
    df1.loc[df1['C_BodyAvg']>= df1['C_Body']  , 'C_LongBody'] = False
    
    
    df1['C_UpShadow'] = df1['high'] - df1['C_BodyHi']
    df1['C_DnShadow'] = df1['C_BodyLo'] - df1['low']
    df1['C_HasUpShadow'] = np.where((df1.C_UpShadow  > (C_ShadowPercent / (100*df1.C_Body))  ),
                                                     True, False)
    df1['C_HasDnShadow'] = np.where((df1.C_DnShadow  > (C_ShadowPercent / (100*df1.C_Body))  ),
                                                     True, False)
    
    df1['C_WhiteBody'] = np.where((df1.open  < df1.close ),True, False)
    df1['C_BlackBody'] = np.where((df1.open  > df1.close ),True, False)
    
    df1['C_Range'] = df1['high']-df1['low']
    df1['C_BodyMiddle']= df1['C_Body'] / (2 + df1['C_BodyLo'])
    
    df1['C_ShadowEquals'] = np.where(  ((df1['C_UpShadow'] == df1['C_DnShadow']) | 
                                 (( abs(df1['C_UpShadow'] - df1['C_DnShadow']) / (df1['C_DnShadow'] * 100) ) )  < 
                                 C_ShadowEqualsPercent) & (((abs(df1['C_DnShadow'] - df1['C_UpShadow']) / (df1['C_UpShadow'] * 100))
                                 < C_ShadowEqualsPercent))  , True, False)
    
    df1['C_IsDojiBody'] = np.where(( (df1['C_Range'] > 0)  & (df1['C_Body'] <= (df1['C_Range'] * (C_DojiBodyPercent / 100))) ), True, False)
    
    df1['C_Doji'] = df1['C_IsDojiBody'] & df1['C_ShadowEquals']
    
    df1['hl2'] = (df1['high'] + df1['low'])/2
    #marking presence and absence of points that constitute harami bearish by checking required conditions
    df1['C_HaramiBearish'] = np.where( ( df1['C_LongBody'].shift(1) & df1['C_WhiteBody'].shift(1) &
                                      df1['C_UpTrend'].shift(1) & df1['C_BlackBody'] & df1['C_SmallBody'] &
                                       (df1['high'] <= df1['C_BodyHi'].shift(1)) &
                                       (df1['low'] >= df1['C_BodyLo'].shift(1)) )   , "1", "0")
  
    return df1


In [60]:
df1HaramiBear = HaramiBear(df1)
print(df1HaramiBear['C_HaramiBearish'].value_counts())

0    11118
1       22
Name: C_HaramiBearish, dtype: int64


A Bullish Harami candlestick is formed when a large bearish red candle appears on Day 1 that is followed by a smaller bearish candle on the next day.

One should note that the important aspect of the bullish Harami is that prices should gap up on Day 2.

In [61]:
def HaramiBull(df):
    df1 = df.copy()
    
    df1['MA100'] = df1.rolling(window=100,min_periods=1 ).mean()['close']
    df1['MA200'] = df1.rolling(window=200,min_periods=1 ).mean()['close']
    df1['MA50'] = df1.rolling(window=50,min_periods=1 ).mean()['close']
    C_DownTrend = True
    C_UpTrend = True
    df1['C_DownTrend'] = np.where((  (df1['close'] < df1['MA50']) & (df1['MA50'] < df1['MA200']) ), True, False)
    df1['C_UpTrend'] = np.where(( (df1['close'] > df1['MA50'])& (df1['MA50'] > df1['MA200']) ), True, False)
    C_Len = 14 #ema depth for bodyAvg
    C_ShadowPercent = 5.0 # size of shadows
    C_ShadowEqualsPercent = 100.0
    C_DojiBodyPercent = 5.0
    C_Factor = 2.0 # shows the number of times the shadow dominates the candlestick body
    df1['C_BodyHi'] = df1[['close','open']].max(axis=1)
    df1['C_BodyLo'] = df1[['close','open']].min(axis=1)
    df1['C_Body'] = df1['C_BodyHi'] - df1['C_BodyLo']
    
    df1['C_BodyAvg'] = pd.Series.ewm(df1['C_Body'], span=C_Len).mean()
    
    
    df1.loc[df1['C_BodyAvg']> df1['C_Body']  , 'C_SmallBody'] = True
    df1.loc[df1['C_BodyAvg']<= df1['C_Body']  , 'C_SmallBody'] = False

    df1.loc[df1['C_BodyAvg']< df1['C_Body']  , 'C_LongBody'] = True
    df1.loc[df1['C_BodyAvg']>= df1['C_Body']  , 'C_LongBody'] = False
    
    
    df1['C_UpShadow'] = df1['high'] - df1['C_BodyHi']
    df1['C_DnShadow'] = df1['C_BodyLo'] - df1['low']
    df1['C_HasUpShadow'] = np.where((df1.C_UpShadow  > (C_ShadowPercent / (100*df1.C_Body))  ),
                                                     True, False)
    df1['C_HasDnShadow'] = np.where((df1.C_DnShadow  > (C_ShadowPercent / (100*df1.C_Body))  ),
                                                     True, False)
    
    df1['C_WhiteBody'] = np.where((df1.open  < df1.close ),True, False)
    df1['C_BlackBody'] = np.where((df1.open  > df1.close ),True, False)
    
    df1['C_Range'] = df1['high']-df1['low']
    df1['C_BodyMiddle']= df1['C_Body'] / (2 + df1['C_BodyLo'])
    
    df1['C_ShadowEquals'] = np.where(  ((df1['C_UpShadow'] == df1['C_DnShadow']) | 
                                 (( abs(df1['C_UpShadow'] - df1['C_DnShadow']) / (df1['C_DnShadow'] * 100) ) )  < 
                                 C_ShadowEqualsPercent) & (((abs(df1['C_DnShadow'] - df1['C_UpShadow']) / (df1['C_UpShadow'] * 100))
                                 < C_ShadowEqualsPercent))  , True, False)
    
    df1['C_IsDojiBody'] = np.where(( (df1['C_Range'] > 0)  & (df1['C_Body'] <= (df1['C_Range'] * (C_DojiBodyPercent / 100))) ), True, False)
    
    df1['C_Doji'] = df1['C_IsDojiBody'] & df1['C_ShadowEquals']
    
    df1['hl2'] = (df1['high'] + df1['low'])/2
   #marking presence and absence of points that constitute harami bullish by checking required conditions
    df1['C_HaramiBullish'] = np.where( ( df1['C_LongBody'].shift(1) & df1['C_BlackBody'].shift(1) &
                                      df1['C_DownTrend'].shift(1) & df1['C_WhiteBody'] & df1['C_SmallBody'] &
                                       (df1['high'] <= df1['C_BodyHi'].shift(1)) &
                                       (df1['low'] >= df1['C_BodyLo'].shift(1)) )   , "1", "0")
  
    return df1


In [62]:
df1HaramiBull = HaramiBull(df1)
print(df1HaramiBull['C_HaramiBullish'].value_counts())

0    11113
1       27
Name: C_HaramiBullish, dtype: int64


A bearish harami cross forms after an uptrend. The first candlestick is a long up candle (typically colored white or green) which shows buyers are in control. This is followed by a doji, which shows indecision on the part of the buyers.  the doji must be contained within the real body of the prior candle. the price drops following the pattern, this confirms the pattern. If the price continues to rise following the doji, the bearish pattern is invalidated.



In [63]:
def HaramiCrossBear(df):
    df1 = df.copy()
    
    df1['MA100'] = df1.rolling(window=100,min_periods=1 ).mean()['close']
    df1['MA200'] = df1.rolling(window=200,min_periods=1 ).mean()['close']
    df1['MA50'] = df1.rolling(window=50,min_periods=1 ).mean()['close']
    C_DownTrend = True
    C_UpTrend = True
    df1['C_DownTrend'] = np.where((  (df1['close'] < df1['MA50']) & (df1['MA50'] < df1['MA200']) ), True, False)
    df1['C_UpTrend'] = np.where(( (df1['close'] > df1['MA50'])& (df1['MA50'] > df1['MA200']) ), True, False)
    C_Len = 14 #ema depth for bodyAvg
    C_ShadowPercent = 5.0 # size of shadows
    C_ShadowEqualsPercent = 100.0
    C_DojiBodyPercent = 5.0
    C_Factor = 2.0 # shows the number of times the shadow dominates the candlestick body
    df1['C_BodyHi'] = df1[['close','open']].max(axis=1)
    df1['C_BodyLo'] = df1[['close','open']].min(axis=1)
    df1['C_Body'] = df1['C_BodyHi'] - df1['C_BodyLo']
    
    df1['C_BodyAvg'] = pd.Series.ewm(df1['C_Body'], span=C_Len).mean()
    
    
    df1.loc[df1['C_BodyAvg']> df1['C_Body']  , 'C_SmallBody'] = True
    df1.loc[df1['C_BodyAvg']<= df1['C_Body']  , 'C_SmallBody'] = False

    df1.loc[df1['C_BodyAvg']< df1['C_Body']  , 'C_LongBody'] = True
    df1.loc[df1['C_BodyAvg']>= df1['C_Body']  , 'C_LongBody'] = False
    
    
    df1['C_UpShadow'] = df1['high'] - df1['C_BodyHi']
    df1['C_DnShadow'] = df1['C_BodyLo'] - df1['low']
    df1['C_HasUpShadow'] = np.where((df1.C_UpShadow  > (C_ShadowPercent / (100*df1.C_Body))  ),
                                                     True, False)
    df1['C_HasDnShadow'] = np.where((df1.C_DnShadow  > (C_ShadowPercent / (100*df1.C_Body))  ),
                                                     True, False)
    
    df1['C_WhiteBody'] = np.where((df1.open  < df1.close ),True, False)
    df1['C_BlackBody'] = np.where((df1.open  > df1.close ),True, False)
    
    df1['C_Range'] = df1['high']-df1['low']
    df1['C_BodyMiddle']= df1['C_Body'] / (2 + df1['C_BodyLo'])
    
    df1['C_ShadowEquals'] = np.where(  ((df1['C_UpShadow'] == df1['C_DnShadow']) | 
                                 (( abs(df1['C_UpShadow'] - df1['C_DnShadow']) / (df1['C_DnShadow'] * 100) ) )  < 
                                 C_ShadowEqualsPercent) & (((abs(df1['C_DnShadow'] - df1['C_UpShadow']) / (df1['C_UpShadow'] * 100))
                                 < C_ShadowEqualsPercent))  , True, False)
    
    df1['C_IsDojiBody'] = np.where(( (df1['C_Range'] > 0)  & (df1['C_Body'] <= (df1['C_Range'] * (C_DojiBodyPercent / 100))) ), True, False)
    
    df1['C_Doji'] = df1['C_IsDojiBody'] & df1['C_ShadowEquals']
    
    df1['hl2'] = (df1['high'] + df1['low'])/2
  #marking presence and absence of points that constitute bearish harami cross by checking required conditions
    df1['C_HaramiCrossBearish'] = np.where( ( df1['C_LongBody'].shift(1) & df1['C_WhiteBody'].shift(1) &
                                      df1['C_UpTrend'].shift(1) & df1['C_IsDojiBody'] &
                                       (df1['high'] <= df1['C_BodyHi'].shift(1)) &
                                       (df1['low'] >= df1['C_BodyLo'].shift(1)) )   , "1", "0")
  
    return df1


In [64]:
df1HaramiCrossBear = HaramiCrossBear(df1)
print(df1HaramiCrossBear['C_HaramiCrossBearish'].value_counts())

0    11139
1        1
Name: C_HaramiCrossBearish, dtype: int64


A bullish harami cross pattern forms after a downtrend. The first candlestick is a long down candle (typically colored black or red) which indicates that the sellers are in control. The second candle, the doji, has a narrow range and opens above the previous day’s close. The doji candlestick closes near to the price it opened at. The doji must be completely contained with the real body of the previous candle.

In [65]:
def HaramiCrossBull(df):
    df1 = df.copy()
    
    df1['MA100'] = df1.rolling(window=100,min_periods=1 ).mean()['close']
    df1['MA200'] = df1.rolling(window=200,min_periods=1 ).mean()['close']
    df1['MA50'] = df1.rolling(window=50,min_periods=1 ).mean()['close']
    C_DownTrend = True
    C_UpTrend = True
    df1['C_DownTrend'] = np.where((  (df1['close'] < df1['MA50']) & (df1['MA50'] < df1['MA200']) ), True, False)
    df1['C_UpTrend'] = np.where(( (df1['close'] > df1['MA50'])& (df1['MA50'] > df1['MA200']) ), True, False)
    C_Len = 14 #ema depth for bodyAvg
    C_ShadowPercent = 5.0 # size of shadows
    C_ShadowEqualsPercent = 100.0
    C_DojiBodyPercent = 5.0
    C_Factor = 2.0 # shows the number of times the shadow dominates the candlestick body
    df1['C_BodyHi'] = df1[['close','open']].max(axis=1)
    df1['C_BodyLo'] = df1[['close','open']].min(axis=1)
    df1['C_Body'] = df1['C_BodyHi'] - df1['C_BodyLo']
    
    df1['C_BodyAvg'] = pd.Series.ewm(df1['C_Body'], span=C_Len).mean()
    
    
    df1.loc[df1['C_BodyAvg']> df1['C_Body']  , 'C_SmallBody'] = True
    df1.loc[df1['C_BodyAvg']<= df1['C_Body']  , 'C_SmallBody'] = False

    df1.loc[df1['C_BodyAvg']< df1['C_Body']  , 'C_LongBody'] = True
    df1.loc[df1['C_BodyAvg']>= df1['C_Body']  , 'C_LongBody'] = False
    
    
    df1['C_UpShadow'] = df1['high'] - df1['C_BodyHi']
    df1['C_DnShadow'] = df1['C_BodyLo'] - df1['low']
    df1['C_HasUpShadow'] = np.where((df1.C_UpShadow  > (C_ShadowPercent / (100*df1.C_Body))  ),
                                                     True, False)
    df1['C_HasDnShadow'] = np.where((df1.C_DnShadow  > (C_ShadowPercent / (100*df1.C_Body))  ),
                                                     True, False)
    
    df1['C_WhiteBody'] = np.where((df1.open  < df1.close ),True, False)
    df1['C_BlackBody'] = np.where((df1.open  > df1.close ),True, False)
    
    df1['C_Range'] = df1['high']-df1['low']
    df1['C_BodyMiddle']= df1['C_Body'] / (2 + df1['C_BodyLo'])
    
    df1['C_ShadowEquals'] = np.where(  ((df1['C_UpShadow'] == df1['C_DnShadow']) | 
                                 (( abs(df1['C_UpShadow'] - df1['C_DnShadow']) / (df1['C_DnShadow'] * 100) ) )  < 
                                 C_ShadowEqualsPercent) & (((abs(df1['C_DnShadow'] - df1['C_UpShadow']) / (df1['C_UpShadow'] * 100))
                                 < C_ShadowEqualsPercent))  , True, False)
    
    df1['C_IsDojiBody'] = np.where(( (df1['C_Range'] > 0)  & (df1['C_Body'] <= (df1['C_Range'] * (C_DojiBodyPercent / 100))) ), True, False)
    
    df1['C_Doji'] = df1['C_IsDojiBody'] & df1['C_ShadowEquals']
    
    df1['hl2'] = (df1['high'] + df1['low'])/2
  #marking presence and absence of points that constitute bullish harami cross by checking required conditions
    df1['C_HaramiCrossBullish'] = np.where( ( df1['C_LongBody'].shift(1) & df1['C_BlackBody'].shift(1) &
                                      df1['C_DownTrend'].shift(1) & df1['C_IsDojiBody']  &
                                       (df1['high'] <= df1['C_BodyHi'].shift(1)) &
                                       (df1['low'] >= df1['C_BodyLo'].shift(1)) )   , "1", "0")
  
    return df1


In [66]:
df1HaramiCrossBull = HaramiCrossBull(df1)
print(df1HaramiCrossBull['C_HaramiCrossBullish'].value_counts())

0    11136
1        4
Name: C_HaramiCrossBullish, dtype: int64


The ‘Inverted Hammer’ gets formed when the price opens at a certain level and then goes much higher..

The price hits a high and then it falls drastically to close near its opening.

The colour of the candle does not matter – it could be either red or green.

It’ll be a red candle if the closing price is lower than the opening price. It’ll be a green candle if the closing price is higher than the opening price. Both are called ‘Inverted Hammer’

In [67]:
def InvertedHammerBull(df):
    df1 = df.copy()
    
    df1['MA100'] = df1.rolling(window=100,min_periods=1 ).mean()['close']
    df1['MA200'] = df1.rolling(window=200,min_periods=1 ).mean()['close']
    df1['MA50'] = df1.rolling(window=50,min_periods=1 ).mean()['close']
    C_DownTrend = True
    C_UpTrend = True
    df1['C_DownTrend'] = np.where((  (df1['close'] < df1['MA50']) & (df1['MA50'] < df1['MA200']) ), True, False)
    df1['C_UpTrend'] = np.where(( (df1['close'] > df1['MA50'])& (df1['MA50'] > df1['MA200']) ), True, False)
    C_Len = 14 #ema depth for bodyAvg
    C_ShadowPercent = 5.0 # size of shadows
    C_ShadowEqualsPercent = 100.0
    C_DojiBodyPercent = 5.0
    C_Factor = 2.0 # shows the number of times the shadow dominates the candlestick body
    df1['C_BodyHi'] = df1[['close','open']].max(axis=1)
    df1['C_BodyLo'] = df1[['close','open']].min(axis=1)
    df1['C_Body'] = df1['C_BodyHi'] - df1['C_BodyLo']
    
    df1['C_BodyAvg'] = pd.Series.ewm(df1['C_Body'], span=C_Len).mean()
    
    
    df1.loc[df1['C_BodyAvg']> df1['C_Body']  , 'C_SmallBody'] = True
    df1.loc[df1['C_BodyAvg']<= df1['C_Body']  , 'C_SmallBody'] = False

    df1.loc[df1['C_BodyAvg']< df1['C_Body']  , 'C_LongBody'] = True
    df1.loc[df1['C_BodyAvg']>= df1['C_Body']  , 'C_LongBody'] = False
    
    
    df1['C_UpShadow'] = df1['high'] - df1['C_BodyHi']
    df1['C_DnShadow'] = df1['C_BodyLo'] - df1['low']
    df1['C_HasUpShadow'] = np.where((df1.C_UpShadow  > (C_ShadowPercent / (100*df1.C_Body))  ),
                                                     True, False)
    df1['C_HasDnShadow'] = np.where((df1.C_DnShadow  > (C_ShadowPercent / (100*df1.C_Body))  ),
                                                     True, False)
    
    df1['C_WhiteBody'] = np.where((df1.open  < df1.close ),True, False)
    df1['C_BlackBody'] = np.where((df1.open  > df1.close ),True, False)
    
    df1['C_Range'] = df1['high']-df1['low']
    df1['C_BodyMiddle']= df1['C_Body'] / (2 + df1['C_BodyLo'])
    
    df1['C_ShadowEquals'] = np.where(  ((df1['C_UpShadow'] == df1['C_DnShadow']) | 
                                 (( abs(df1['C_UpShadow'] - df1['C_DnShadow']) / (df1['C_DnShadow'] * 100) ) )  < 
                                 C_ShadowEqualsPercent) & (((abs(df1['C_DnShadow'] - df1['C_UpShadow']) / (df1['C_UpShadow'] * 100))
                                 < C_ShadowEqualsPercent))  , True, False)
    
    df1['C_IsDojiBody'] = np.where(( (df1['C_Range'] > 0)  & (df1['C_Body'] <= (df1['C_Range'] * (C_DojiBodyPercent / 100))) ), True, False)
    
    df1['C_Doji'] = df1['C_IsDojiBody'] & df1['C_ShadowEquals']
    
    df1['hl2'] = (df1['high'] + df1['low'])/2
  #marking presence and absence of points that constitute bullish inverted hammer cross by checking required conditions
    df1['C_InvertedHammerBullish'] = np.where( ( df1['C_SmallBody'] & (df1['C_Body'] > 0) &
                                              (df1['C_BodyHi'] < df1['hl2']) &
                                              (df1['C_UpShadow'] >= C_Factor * df1['C_Body']) &
                                               (~ df1['C_HasDnShadow']))   , "1", "0")
  
    return df1


In [68]:
df1InvertedHammerBull = InvertedHammerBull(df1)
print(df1InvertedHammerBull['C_InvertedHammerBullish'].value_counts())

0    10199
1      941
Name: C_InvertedHammerBullish, dtype: int64


The long lower shadow candlestick is a technical indicator that is used by traders to identify a reversal in the market trends. 
First, a candlestick appears This candle can be white (or green) or black (or red). Second, that candlestick must have a long lower tail. To be more specific, the lower shadow should comprise two-thirds or more of the total range of the candlestick (from the top of the upper shadow to the bottom of the lower shadow).

In [69]:
def LongLowerShadowBull(df):
    df1 = df.copy()
    
    df1['MA100'] = df1.rolling(window=100,min_periods=1 ).mean()['close']
    df1['MA200'] = df1.rolling(window=200,min_periods=1 ).mean()['close']
    df1['MA50'] = df1.rolling(window=50,min_periods=1 ).mean()['close']
    C_DownTrend = True
    C_UpTrend = True
    df1['C_DownTrend'] = np.where((  (df1['close'] < df1['MA50']) & (df1['MA50'] < df1['MA200']) ), True, False)
    df1['C_UpTrend'] = np.where(( (df1['close'] > df1['MA50'])& (df1['MA50'] > df1['MA200']) ), True, False)
    C_Len = 14 #ema depth for bodyAvg
    C_ShadowPercent = 5.0 # size of shadows
    C_ShadowEqualsPercent = 100.0
    C_DojiBodyPercent = 5.0
    C_Factor = 2.0 # shows the number of times the shadow dominates the candlestick body
    df1['C_BodyHi'] = df1[['close','open']].max(axis=1)
    df1['C_BodyLo'] = df1[['close','open']].min(axis=1)
    df1['C_Body'] = df1['C_BodyHi'] - df1['C_BodyLo']
    
    df1['C_BodyAvg'] = pd.Series.ewm(df1['C_Body'], span=C_Len).mean()
    
    
    df1.loc[df1['C_BodyAvg']> df1['C_Body']  , 'C_SmallBody'] = True
    df1.loc[df1['C_BodyAvg']<= df1['C_Body']  , 'C_SmallBody'] = False

    df1.loc[df1['C_BodyAvg']< df1['C_Body']  , 'C_LongBody'] = True
    df1.loc[df1['C_BodyAvg']>= df1['C_Body']  , 'C_LongBody'] = False
    
    
    df1['C_UpShadow'] = df1['high'] - df1['C_BodyHi']
    df1['C_DnShadow'] = df1['C_BodyLo'] - df1['low']
    df1['C_HasUpShadow'] = np.where((df1.C_UpShadow  > (C_ShadowPercent / (100*df1.C_Body))  ),
                                                     True, False)
    df1['C_HasDnShadow'] = np.where((df1.C_DnShadow  > (C_ShadowPercent / (100*df1.C_Body))  ),
                                                     True, False)
    
    df1['C_WhiteBody'] = np.where((df1.open  < df1.close ),True, False)
    df1['C_BlackBody'] = np.where((df1.open  > df1.close ),True, False)
    
    df1['C_Range'] = df1['high']-df1['low']
    df1['C_BodyMiddle']= df1['C_Body'] / (2 + df1['C_BodyLo'])
    
    df1['C_ShadowEquals'] = np.where(  ((df1['C_UpShadow'] == df1['C_DnShadow']) | 
                                 (( abs(df1['C_UpShadow'] - df1['C_DnShadow']) / (df1['C_DnShadow'] * 100) ) )  < 
                                 C_ShadowEqualsPercent) & (((abs(df1['C_DnShadow'] - df1['C_UpShadow']) / (df1['C_UpShadow'] * 100))
                                 < C_ShadowEqualsPercent))  , True, False)
    
    df1['C_IsDojiBody'] = np.where(( (df1['C_Range'] > 0)  & (df1['C_Body'] <= (df1['C_Range'] * (C_DojiBodyPercent / 100))) ), True, False)
    
    df1['C_Doji'] = df1['C_IsDojiBody'] & df1['C_ShadowEquals']
    
    df1['hl2'] = (df1['high'] + df1['low'])/2
    
    C_LongLowerShadowPercent = 75.0
    #marking long lower shadow data points based on condition satisfaction
    df1['C_LongLowerShadowBullish'] = np.where( ( df1['C_DnShadow'] > (df1['C_Range'] / (100*C_LongLowerShadowPercent)) )   , "1", "0")
  
    return df1


In [70]:
df1LongLowerShadowBull = LongLowerShadowBull(df1)
print(df1LongLowerShadowBull['C_LongLowerShadowBullish'].value_counts())

1    9192
0    1948
Name: C_LongLowerShadowBullish, dtype: int64


Long upper shadow:
A  bearish candlestick whereby the length of the tail / lower shadow is at least two thirds of the total shadow length.

In [71]:
def LongUpperShadowBearish(df):
    df1 = df.copy()
    
    df1['MA100'] = df1.rolling(window=100,min_periods=1 ).mean()['close']
    df1['MA200'] = df1.rolling(window=200,min_periods=1 ).mean()['close']
    df1['MA50'] = df1.rolling(window=50,min_periods=1 ).mean()['close']
    C_DownTrend = True
    C_UpTrend = True
    df1['C_DownTrend'] = np.where((  (df1['close'] < df1['MA50']) & (df1['MA50'] < df1['MA200']) ), True, False)
    df1['C_UpTrend'] = np.where(( (df1['close'] > df1['MA50'])& (df1['MA50'] > df1['MA200']) ), True, False)
    C_Len = 14 #ema depth for bodyAvg
    C_ShadowPercent = 5.0 # size of shadows
    C_ShadowEqualsPercent = 100.0
    C_DojiBodyPercent = 5.0
    C_Factor = 2.0 # shows the number of times the shadow dominates the candlestick body
    df1['C_BodyHi'] = df1[['close','open']].max(axis=1)
    df1['C_BodyLo'] = df1[['close','open']].min(axis=1)
    df1['C_Body'] = df1['C_BodyHi'] - df1['C_BodyLo']
    
    df1['C_BodyAvg'] = pd.Series.ewm(df1['C_Body'], span=C_Len).mean()
    
    
    df1.loc[df1['C_BodyAvg']> df1['C_Body']  , 'C_SmallBody'] = True
    df1.loc[df1['C_BodyAvg']<= df1['C_Body']  , 'C_SmallBody'] = False

    df1.loc[df1['C_BodyAvg']< df1['C_Body']  , 'C_LongBody'] = True
    df1.loc[df1['C_BodyAvg']>= df1['C_Body']  , 'C_LongBody'] = False
    
    
    df1['C_UpShadow'] = df1['high'] - df1['C_BodyHi']
    df1['C_DnShadow'] = df1['C_BodyLo'] - df1['low']
    df1['C_HasUpShadow'] = np.where((df1.C_UpShadow  > (C_ShadowPercent / (100*df1.C_Body))  ),
                                                     True, False)
    df1['C_HasDnShadow'] = np.where((df1.C_DnShadow  > (C_ShadowPercent / (100*df1.C_Body))  ),
                                                     True, False)
    
    df1['C_WhiteBody'] = np.where((df1.open  < df1.close ),True, False)
    df1['C_BlackBody'] = np.where((df1.open  > df1.close ),True, False)
    
    df1['C_Range'] = df1['high']-df1['low']
    df1['C_BodyMiddle']= df1['C_Body'] / (2 + df1['C_BodyLo'])
    
    df1['C_ShadowEquals'] = np.where(  ((df1['C_UpShadow'] == df1['C_DnShadow']) | 
                                 (( abs(df1['C_UpShadow'] - df1['C_DnShadow']) / (df1['C_DnShadow'] * 100) ) )  < 
                                 C_ShadowEqualsPercent) & (((abs(df1['C_DnShadow'] - df1['C_UpShadow']) / (df1['C_UpShadow'] * 100))
                                 < C_ShadowEqualsPercent))  , True, False)
    
    df1['C_IsDojiBody'] = np.where(( (df1['C_Range'] > 0)  & (df1['C_Body'] <= (df1['C_Range'] * (C_DojiBodyPercent / 100))) ), True, False)
    
    df1['C_Doji'] = df1['C_IsDojiBody'] & df1['C_ShadowEquals']
    
    df1['hl2'] = (df1['high'] + df1['low'])/2
    
    C_LongLowerShadowPercent = 75.0
  #marking long upper shadow data points based on condition satisfaction
    df1['C_LongUpperShadowBearish'] = np.where( ( df1['C_UpShadow'] > (df1['C_Range'] / (100*C_LongLowerShadowPercent)) )   , "1", "0")
  
    return df1


In [72]:
df1LongUpperShadowBearish = LongUpperShadowBearish(df1)
print(df1LongUpperShadowBearish['C_LongUpperShadowBearish'].value_counts())

1    9223
0    1917
Name: C_LongUpperShadowBearish, dtype: int64


A black marubozu candle has a long black body and is formed when the open equals the high and the close equals the low. A black marubozu indicates that sellers controlled the price from the opening bell to the close of the day, and is considered very bearish.

In [73]:
def MarubozuBlackBearish(df):
    df1 = df.copy()
    
    df1['MA100'] = df1.rolling(window=100,min_periods=1 ).mean()['close']
    df1['MA200'] = df1.rolling(window=200,min_periods=1 ).mean()['close']
    df1['MA50'] = df1.rolling(window=50,min_periods=1 ).mean()['close']
    C_DownTrend = True
    C_UpTrend = True
    df1['C_DownTrend'] = np.where((  (df1['close'] < df1['MA50']) & (df1['MA50'] < df1['MA200']) ), True, False)
    df1['C_UpTrend'] = np.where(( (df1['close'] > df1['MA50'])& (df1['MA50'] > df1['MA200']) ), True, False)
    C_Len = 14 #ema depth for bodyAvg
    C_ShadowPercent = 5.0 # size of shadows
    C_ShadowEqualsPercent = 100.0
    C_DojiBodyPercent = 5.0
    C_Factor = 2.0 # shows the number of times the shadow dominates the candlestick body
    df1['C_BodyHi'] = df1[['close','open']].max(axis=1)
    df1['C_BodyLo'] = df1[['close','open']].min(axis=1)
    df1['C_Body'] = df1['C_BodyHi'] - df1['C_BodyLo']
    
    df1['C_BodyAvg'] = pd.Series.ewm(df1['C_Body'], span=C_Len).mean()
    
    
    df1.loc[df1['C_BodyAvg']> df1['C_Body']  , 'C_SmallBody'] = True
    df1.loc[df1['C_BodyAvg']<= df1['C_Body']  , 'C_SmallBody'] = False

    df1.loc[df1['C_BodyAvg']< df1['C_Body']  , 'C_LongBody'] = True
    df1.loc[df1['C_BodyAvg']>= df1['C_Body']  , 'C_LongBody'] = False
    
    
    df1['C_UpShadow'] = df1['high'] - df1['C_BodyHi']
    df1['C_DnShadow'] = df1['C_BodyLo'] - df1['low']
    df1['C_HasUpShadow'] = np.where((df1.C_UpShadow  > (C_ShadowPercent / (100*df1.C_Body))  ),
                                                     True, False)
    df1['C_HasDnShadow'] = np.where((df1.C_DnShadow  > (C_ShadowPercent / (100*df1.C_Body))  ),
                                                     True, False)
    
    df1['C_WhiteBody'] = np.where((df1.open  < df1.close ),True, False)
    df1['C_BlackBody'] = np.where((df1.open  > df1.close ),True, False)
    
    df1['C_Range'] = df1['high']-df1['low']
    df1['C_BodyMiddle']= df1['C_Body'] / (2 + df1['C_BodyLo'])
    
    df1['C_ShadowEquals'] = np.where(  ((df1['C_UpShadow'] == df1['C_DnShadow']) | 
                                 (( abs(df1['C_UpShadow'] - df1['C_DnShadow']) / (df1['C_DnShadow'] * 100) ) )  < 
                                 C_ShadowEqualsPercent) & (((abs(df1['C_DnShadow'] - df1['C_UpShadow']) / (df1['C_UpShadow'] * 100))
                                 < C_ShadowEqualsPercent))  , True, False)
    
    df1['C_IsDojiBody'] = np.where(( (df1['C_Range'] > 0)  & (df1['C_Body'] <= (df1['C_Range'] * (C_DojiBodyPercent / 100))) ), True, False)
    
    df1['C_Doji'] = df1['C_IsDojiBody'] & df1['C_ShadowEquals']
    
    df1['hl2'] = (df1['high'] + df1['low'])/2
    
    C_MarubozuShadowPercentBearish = 5.0
 #marking marubozu black shadow data points based on condition satisfaction
    df1['C_MarubozuBlackBearish'] = np.where( ( df1['C_BlackBody'] & df1['C_LongBody'] &
                                           (df1['C_UpShadow'] <= C_MarubozuShadowPercentBearish/(100*df1['C_Body']) ) &
                                            (df1['C_DnShadow'] <= C_MarubozuShadowPercentBearish/(100*df1['C_Body'])) &
                                            df1['C_BlackBody']  )   , "1", "0")
  
    return df1


In [74]:
df1MarubozuBlackBearish = MarubozuBlackBearish(df1)
print(df1MarubozuBlackBearish['C_MarubozuBlackBearish'].value_counts())

0    9878
1    1262
Name: C_MarubozuBlackBearish, dtype: int64


A White Marubozu is a one day bullish indicator that moves upward and is considered very bullish. If a White Marubozu occurs at the end of an uptrend, a continuation is likely. If a White Marubozu occurs at the end of a downtrend, a reversal is likely.

In [75]:
def MarubozuWhiteBullish(df):
    df1 = df.copy()
    
    df1['MA100'] = df1.rolling(window=100,min_periods=1 ).mean()['close']
    df1['MA200'] = df1.rolling(window=200,min_periods=1 ).mean()['close']
    df1['MA50'] = df1.rolling(window=50,min_periods=1 ).mean()['close']
    C_DownTrend = True
    C_UpTrend = True
    df1['C_DownTrend'] = np.where((  (df1['close'] < df1['MA50']) & (df1['MA50'] < df1['MA200']) ), True, False)
    df1['C_UpTrend'] = np.where(( (df1['close'] > df1['MA50'])& (df1['MA50'] > df1['MA200']) ), True, False)
    C_Len = 14 #ema depth for bodyAvg
    C_ShadowPercent = 5.0 # size of shadows
    C_ShadowEqualsPercent = 100.0
    C_DojiBodyPercent = 5.0
    C_Factor = 2.0 # shows the number of times the shadow dominates the candlestick body
    df1['C_BodyHi'] = df1[['close','open']].max(axis=1)
    df1['C_BodyLo'] = df1[['close','open']].min(axis=1)
    df1['C_Body'] = df1['C_BodyHi'] - df1['C_BodyLo']
    
    df1['C_BodyAvg'] = pd.Series.ewm(df1['C_Body'], span=C_Len).mean()
    
    
    df1.loc[df1['C_BodyAvg']> df1['C_Body']  , 'C_SmallBody'] = True
    df1.loc[df1['C_BodyAvg']<= df1['C_Body']  , 'C_SmallBody'] = False

    df1.loc[df1['C_BodyAvg']< df1['C_Body']  , 'C_LongBody'] = True
    df1.loc[df1['C_BodyAvg']>= df1['C_Body']  , 'C_LongBody'] = False
    
    
    df1['C_UpShadow'] = df1['high'] - df1['C_BodyHi']
    df1['C_DnShadow'] = df1['C_BodyLo'] - df1['low']
    df1['C_HasUpShadow'] = np.where((df1.C_UpShadow  > (C_ShadowPercent / (100*df1.C_Body))  ),
                                                     True, False)
    df1['C_HasDnShadow'] = np.where((df1.C_DnShadow  > (C_ShadowPercent / (100*df1.C_Body))  ),
                                                     True, False)
    
    df1['C_WhiteBody'] = np.where((df1.open  < df1.close ),True, False)
    df1['C_BlackBody'] = np.where((df1.open  > df1.close ),True, False)
    
    df1['C_Range'] = df1['high']-df1['low']
    df1['C_BodyMiddle']= df1['C_Body'] / (2 + df1['C_BodyLo'])
    
    df1['C_ShadowEquals'] = np.where(  ((df1['C_UpShadow'] == df1['C_DnShadow']) | 
                                 (( abs(df1['C_UpShadow'] - df1['C_DnShadow']) / (df1['C_DnShadow'] * 100) ) )  < 
                                 C_ShadowEqualsPercent) & (((abs(df1['C_DnShadow'] - df1['C_UpShadow']) / (df1['C_UpShadow'] * 100))
                                 < C_ShadowEqualsPercent))  , True, False)
    
    df1['C_IsDojiBody'] = np.where(( (df1['C_Range'] > 0)  & (df1['C_Body'] <= (df1['C_Range'] * (C_DojiBodyPercent / 100))) ), True, False)
    
    df1['C_Doji'] = df1['C_IsDojiBody'] & df1['C_ShadowEquals']
    
    df1['hl2'] = (df1['high'] + df1['low'])/2
    
    C_MarubozuShadowPercentWhite = 5.0
 #marking marubozu white data points based on condition satisfaction
    df1['MarubozuWhiteBullish'] = np.where( ( df1['C_WhiteBody'] & df1['C_LongBody'] &
                                           (df1['C_UpShadow'] <= C_MarubozuShadowPercentWhite/(100*df1['C_Body']) ) &
                                            (df1['C_DnShadow'] <= C_MarubozuShadowPercentWhite/(100*df1['C_Body'])) &
                                            df1['C_WhiteBody']  )   , "1", "0")
  
    return df1


In [76]:
df1MarubozuWhiteBullish =MarubozuWhiteBullish(df1)
print(df1MarubozuWhiteBullish['MarubozuWhiteBullish'].value_counts())

0    10173
1      967
Name: MarubozuWhiteBullish, dtype: int64


First candle:
a candle in a downtrend
black body
Second candle:
a doji candle
a doji body below the previous candle body
the high price above the previous candle low price
Third candle:
white body
candle body above the previous candle body
the closing price above the midpoint of the first candle body

In [77]:
def MorningDojiStarBullish(df):
    
    df1 = df.copy()
    
    df1['MA100'] = df.rolling(window=100,min_periods=1 ).mean()['close']
    df1['MA200'] = df.rolling(window=200,min_periods=1 ).mean()['close']
    df1['MA50'] = df.rolling(window=50,min_periods=1 ).mean()['close']
    C_DownTrend = True
    C_UpTrend = True
    df1['C_DownTrend'] = np.where((  (df1['close'] < df1['MA50']) & (df1['MA50'] < df1['MA200']) ), True, False)
    df1['C_UpTrend'] = np.where(( (df1['close'] > df1['MA50'])& (df1['MA50'] > df1['MA200']) ), True, False)
    C_Len = 14 #ema depth for bodyAvg
    C_ShadowPercent = 5.0 # size of shadows
    C_ShadowEqualsPercent = 100.0
    C_DojiBodyPercent = 5.0
    C_Factor = 2.0 # shows the number of times the shadow dominates the candlestick body
    df1['C_BodyHi'] = df1[['close','open']].max(axis=1)
    df1['C_BodyLo'] = df1[['close','open']].min(axis=1)
    df1['C_Body'] = df1['C_BodyHi'] - df1['C_BodyLo']
    
    df1['C_BodyAvg'] = pd.Series.ewm(df1['C_Body'], span=C_Len).mean()
    
    
    df1.loc[df1['C_BodyAvg']> df1['C_Body']  , 'C_SmallBody'] = True
    df1.loc[df1['C_BodyAvg']<= df1['C_Body']  , 'C_SmallBody'] = False

    df1.loc[df1['C_BodyAvg']< df1['C_Body']  , 'C_LongBody'] = True
    df1.loc[df1['C_BodyAvg']>= df1['C_Body']  , 'C_LongBody'] = False
    
    
    df1['C_UpShadow'] = df1['high'] - df1['C_BodyHi']
    df1['C_DnShadow'] = df1['C_BodyLo'] - df1['low']
    df1['C_HasUpShadow'] = np.where((df1.C_UpShadow  > (C_ShadowPercent / (100*df1.C_Body))  ),
                                                     True, False)
    df1['C_HasDnShadow'] = np.where((df1.C_DnShadow  > (C_ShadowPercent / (100*df1.C_Body))  ),
                                                     True, False)
    
    df1['C_WhiteBody'] = np.where((df1.open  < df1.close ),True, False)
    df1['C_BlackBody'] = np.where((df1.open  > df1.close ),True, False)
    
    df1['C_Range'] = df1['high']-df1['low']
    df1['C_BodyMiddle']= df1['C_Body'] / (2 + df1['C_BodyLo'])
    
    df1['C_ShadowEquals'] = np.where(  ((df1['C_UpShadow'] == df1['C_DnShadow']) | 
                                 (( abs(df1['C_UpShadow'] - df1['C_DnShadow']) / (df1['C_DnShadow'] * 100) ) )  < 
                                 C_ShadowEqualsPercent) & (((abs(df1['C_DnShadow'] - df1['C_UpShadow']) / (df1['C_UpShadow'] * 100))
                                 < C_ShadowEqualsPercent))  , True, False)
    
    df1['C_IsDojiBody'] = np.where(( (df1['C_Range'] > 0)  & (df1['C_Body'] <= (df1['C_Range'] * (C_DojiBodyPercent / 100))) ), True, False)
    
    df1['C_Doji'] = df1['C_IsDojiBody'] & df1['C_ShadowEquals']
    
    df1['hl2'] = (df['high'] + df1['low'])/2

 #marking morning doji star data points based on condition satisfaction
    df1['C_MorningDojiStarBullish'] = np.where(( df1['C_LongBody'].shift(2) & df1['C_IsDojiBody'].shift(1) &
                                            df1['C_LongBody'] & df1['C_DownTrend'] &  df1['C_BlackBody'].shift(2) &
                                             (df1['C_BodyHi'].shift(1) < df1['C_BodyLo'].shift(2)) &
                                              df1['C_WhiteBody'] &  (df1['C_BodyHi'] >= df1['C_BodyMiddle'].shift(2)) &
                                              (df1['C_BodyHi'] < df1['C_BodyHi'].shift(2)) & 
                                               (df1['C_BodyHi'].shift(1) < df1['C_BodyLo'])), "1", "0" )
    
    return df1

In [78]:
df1MorningDojiStarBullish =MorningDojiStarBullish(df1)
print(df1MorningDojiStarBullish['C_MorningDojiStarBullish'].value_counts())

0    11140
Name: C_MorningDojiStarBullish, dtype: int64


A morning star is a visual pattern consisting of three candlesticks that are interpreted as bullish signs by technical analysts. A morning star forms following a downward trend and it indicates the start of an upward climb. It is a sign of a reversal in the previous price trend.A morning star is a visual pattern, so there are no particular calculations to perform. A morning star is a three-candle pattern with the low point on the second candle. However, the low point is only apparent after the close of the third candle.

A morning star is a three-candle pattern with the low point on the second candle. However, the low point is only apparent after the close of the third candle.

Other technical indicators can help predict if a morning star is forming, such as whether the price action is nearing a support zone or whether or not the relative strength indicator (RSI) is showing that the stock or commodity is oversold.
1



In [79]:
def MorningStarBullish(df):
    
    df1 = df.copy()
    
    df1['MA100'] = df.rolling(window=100,min_periods=1 ).mean()['close']
    df1['MA200'] = df.rolling(window=200,min_periods=1 ).mean()['close']
    df1['MA50'] = df.rolling(window=50,min_periods=1 ).mean()['close']
    C_DownTrend = True
    C_UpTrend = True
    df1['C_DownTrend'] = np.where((  (df1['close'] < df1['MA50']) & (df1['MA50'] < df1['MA200']) ), True, False)
    df1['C_UpTrend'] = np.where(( (df1['close'] > df1['MA50'])& (df1['MA50'] > df1['MA200']) ), True, False)
    C_Len = 14 #ema depth for bodyAvg
    C_ShadowPercent = 5.0 # size of shadows
    C_ShadowEqualsPercent = 100.0
    C_DojiBodyPercent = 5.0
    C_Factor = 2.0 # shows the number of times the shadow dominates the candlestick body
    df1['C_BodyHi'] = df1[['close','open']].max(axis=1)
    df1['C_BodyLo'] = df1[['close','open']].min(axis=1)
    df1['C_Body'] = df1['C_BodyHi'] - df1['C_BodyLo']
    
    df1['C_BodyAvg'] = pd.Series.ewm(df1['C_Body'], span=C_Len).mean()
    
    
    df1.loc[df1['C_BodyAvg']> df1['C_Body']  , 'C_SmallBody'] = True
    df1.loc[df1['C_BodyAvg']<= df1['C_Body']  , 'C_SmallBody'] = False

    df1.loc[df1['C_BodyAvg']< df1['C_Body']  , 'C_LongBody'] = True
    df1.loc[df1['C_BodyAvg']>= df1['C_Body']  , 'C_LongBody'] = False
    
    
    df1['C_UpShadow'] = df1['high'] - df1['C_BodyHi']
    df1['C_DnShadow'] = df1['C_BodyLo'] - df1['low']
    df1['C_HasUpShadow'] = np.where((df1.C_UpShadow  > (C_ShadowPercent / (100*df1.C_Body))  ),
                                                     True, False)
    df1['C_HasDnShadow'] = np.where((df1.C_DnShadow  > (C_ShadowPercent / (100*df1.C_Body))  ),
                                                     True, False)
    
    df1['C_WhiteBody'] = np.where((df1.open  < df1.close ),True, False)
    df1['C_BlackBody'] = np.where((df1.open  > df1.close ),True, False)
    
    df1['C_Range'] = df1['high']-df1['low']
    df1['C_BodyMiddle']= df1['C_Body'] / (2 + df1['C_BodyLo'])
    
    df1['C_ShadowEquals'] = np.where(  ((df1['C_UpShadow'] == df1['C_DnShadow']) | 
                                 (( abs(df1['C_UpShadow'] - df1['C_DnShadow']) / (df1['C_DnShadow'] * 100) ) )  < 
                                 C_ShadowEqualsPercent) & (((abs(df1['C_DnShadow'] - df1['C_UpShadow']) / (df1['C_UpShadow'] * 100))
                                 < C_ShadowEqualsPercent))  , True, False)
    
    df1['C_IsDojiBody'] = np.where(( (df1['C_Range'] > 0)  & (df1['C_Body'] <= (df1['C_Range'] * (C_DojiBodyPercent / 100))) ), True, False)
    
    df1['C_Doji'] = df1['C_IsDojiBody'] & df1['C_ShadowEquals']
    
    df1['hl2'] = (df['high'] + df1['low'])/2
 #marking morning  star data points based on condition satisfaction
    df1['C_MorningStarBullish'] = np.where(( df1['C_LongBody'].shift(2) & df1['C_SmallBody'].shift(1) &
                                            df1['C_LongBody'] & df1['C_DownTrend'] &  df1['C_BlackBody'].shift(2) &
                                             (df1['C_BodyHi'].shift(1) < df1['C_BodyLo'].shift(2)) &
                                              df1['C_WhiteBody'] &  (df1['C_BodyHi'] >= df1['C_BodyMiddle'].shift(2)) &
                                              (df1['C_BodyHi'] < df1['C_BodyHi'].shift(2)) & 
                                               (df1['C_BodyHi'].shift(1) < df1['C_BodyLo'])), "1", "0" )
    
    return df1

In [80]:
df1MorningStarBullish =MorningStarBullish(df1)
print(df1MorningStarBullish['C_MorningStarBullish'].value_counts())

0    11140
Name: C_MorningStarBullish, dtype: int64


The on neck pattern occurs during a downtrend, or a pullback within an uptrend, when a bearish candle with a long real body is followed by a smaller bullish candle that fails to close above the bearish candle's close.

In [81]:
def OnNeckBearish(df):
    
    df1 = df.copy()
    
    df1['MA100'] = df.rolling(window=100,min_periods=1 ).mean()['close']
    df1['MA200'] = df.rolling(window=200,min_periods=1 ).mean()['close']
    df1['MA50'] = df.rolling(window=50,min_periods=1 ).mean()['close']
    C_DownTrend = True
    C_UpTrend = True
    df1['C_DownTrend'] = np.where((  (df1['close'] < df1['MA50']) & (df1['MA50'] < df1['MA200']) ), True, False)
    df1['C_UpTrend'] = np.where(( (df1['close'] > df1['MA50'])& (df1['MA50'] > df1['MA200']) ), True, False)
    C_Len = 14 #ema depth for bodyAvg
    C_ShadowPercent = 5.0 # size of shadows
    C_ShadowEqualsPercent = 100.0
    C_DojiBodyPercent = 5.0
    C_Factor = 2.0 # shows the number of times the shadow dominates the candlestick body
    df1['C_BodyHi'] = df1[['close','open']].max(axis=1)
    df1['C_BodyLo'] = df1[['close','open']].min(axis=1)
    df1['C_Body'] = df1['C_BodyHi'] - df1['C_BodyLo']
    
    df1['C_BodyAvg'] = pd.Series.ewm(df1['C_Body'], span=C_Len).mean()
    
    
    df1.loc[df1['C_BodyAvg']> df1['C_Body']  , 'C_SmallBody'] = True
    df1.loc[df1['C_BodyAvg']<= df1['C_Body']  , 'C_SmallBody'] = False

    df1.loc[df1['C_BodyAvg']< df1['C_Body']  , 'C_LongBody'] = True
    df1.loc[df1['C_BodyAvg']>= df1['C_Body']  , 'C_LongBody'] = False
    
    
    df1['C_UpShadow'] = df1['high'] - df1['C_BodyHi']
    df1['C_DnShadow'] = df1['C_BodyLo'] - df1['low']
    df1['C_HasUpShadow'] = np.where((df1.C_UpShadow  > (C_ShadowPercent / (100*df1.C_Body))  ),
                                                     True, False)
    df1['C_HasDnShadow'] = np.where((df1.C_DnShadow  > (C_ShadowPercent / (100*df1.C_Body))  ),
                                                     True, False)
    
    df1['C_WhiteBody'] = np.where((df1.open  < df1.close ),True, False)
    df1['C_BlackBody'] = np.where((df1.open  > df1.close ),True, False)
    
    df1['C_Range'] = df1['high']-df1['low']
    df1['C_BodyMiddle']= df1['C_Body'] / (2 + df1['C_BodyLo'])
    
    df1['C_ShadowEquals'] = np.where(  ((df1['C_UpShadow'] == df1['C_DnShadow']) | 
                                 (( abs(df1['C_UpShadow'] - df1['C_DnShadow']) / (df1['C_DnShadow'] * 100) ) )  < 
                                 C_ShadowEqualsPercent) & (((abs(df1['C_DnShadow'] - df1['C_UpShadow']) / (df1['C_UpShadow'] * 100))
                                 < C_ShadowEqualsPercent))  , True, False)
    
    df1['C_IsDojiBody'] = np.where(( (df1['C_Range'] > 0)  & (df1['C_Body'] <= (df1['C_Range'] * (C_DojiBodyPercent / 100))) ), True, False)
    
    df1['C_Doji'] = df1['C_IsDojiBody'] & df1['C_ShadowEquals']
    
    df1['hl2'] = (df['high'] + df1['low'])/2
 #marking on neck data points based on condition satisfaction
    df1['C_OnNeckBearish'] = np.where((df1['C_DownTrend'] & df1['C_BlackBody'].shift(1) &
                                     df1['C_LongBody'].shift(1) & df1['C_WhiteBody'] & 
                                      (df1['open'] < df1['close'].shift(1)) &
                                     df1['C_SmallBody'] & ( df1['C_Range'] != 0 ) &
                                     (abs(df1['close'] - df1['low'].shift(1)) <= df1['C_BodyAvg']*0.05) ), "1", "0" )
    
    return df1

In [82]:
df1OnNeckBearish =OnNeckBearish(df1)
print(df1OnNeckBearish['C_OnNeckBearish'].value_counts())

0    11140
Name: C_OnNeckBearish, dtype: int64


Piercing pattern :
The pattern is preceded by a downward trend in price. (This may be only a short downtrend, but if the candles appear after an upward trend in price it is not an important reversal indicator).
The price gaps lower to begin the second day. (This pattern is mostly found in stocks because of their ability to have overnight gaps unlike currencies or other 24-hour trading assets. This pattern may occur in any asset class on a weekly chart, however).
The second candle must close above the midpoint of the first candle. (This signifies that buyers overwhelmed sellers on this day.)

In [83]:
def PiercingBullish(df):
    
    df1 = df.copy()
    
    df1['MA100'] = df.rolling(window=100,min_periods=1 ).mean()['close']
    df1['MA200'] = df.rolling(window=200,min_periods=1 ).mean()['close']
    df1['MA50'] = df.rolling(window=50,min_periods=1 ).mean()['close']
    C_DownTrend = True
    C_UpTrend = True
    df1['C_DownTrend'] = np.where((  (df1['close'] < df1['MA50']) & (df1['MA50'] < df1['MA200']) ), True, False)
    df1['C_UpTrend'] = np.where(( (df1['close'] > df1['MA50'])& (df1['MA50'] > df1['MA200']) ), True, False)
    C_Len = 14 #ema depth for bodyAvg
    C_ShadowPercent = 5.0 # size of shadows
    C_ShadowEqualsPercent = 100.0
    C_DojiBodyPercent = 5.0
    C_Factor = 2.0 # shows the number of times the shadow dominates the candlestick body
    df1['C_BodyHi'] = df1[['close','open']].max(axis=1)
    df1['C_BodyLo'] = df1[['close','open']].min(axis=1)
    df1['C_Body'] = df1['C_BodyHi'] - df1['C_BodyLo']
    
    df1['C_BodyAvg'] = pd.Series.ewm(df1['C_Body'], span=C_Len).mean()
    
    
    df1.loc[df1['C_BodyAvg']> df1['C_Body']  , 'C_SmallBody'] = True
    df1.loc[df1['C_BodyAvg']<= df1['C_Body']  , 'C_SmallBody'] = False

    df1.loc[df1['C_BodyAvg']< df1['C_Body']  , 'C_LongBody'] = True
    df1.loc[df1['C_BodyAvg']>= df1['C_Body']  , 'C_LongBody'] = False
    
    
    df1['C_UpShadow'] = df1['high'] - df1['C_BodyHi']
    df1['C_DnShadow'] = df1['C_BodyLo'] - df1['low']
    df1['C_HasUpShadow'] = np.where((df1.C_UpShadow  > (C_ShadowPercent / (100*df1.C_Body))  ),
                                                     True, False)
    df1['C_HasDnShadow'] = np.where((df1.C_DnShadow  > (C_ShadowPercent / (100*df1.C_Body))  ),
                                                     True, False)
    
    df1['C_WhiteBody'] = np.where((df1.open  < df1.close ),True, False)
    df1['C_BlackBody'] = np.where((df1.open  > df1.close ),True, False)
    
    df1['C_Range'] = df1['high']-df1['low']
    df1['C_BodyMiddle']= df1['C_Body'] / (2 + df1['C_BodyLo'])
    
    df1['C_ShadowEquals'] = np.where(  ((df1['C_UpShadow'] == df1['C_DnShadow']) | 
                                 (( abs(df1['C_UpShadow'] - df1['C_DnShadow']) / (df1['C_DnShadow'] * 100) ) )  < 
                                 C_ShadowEqualsPercent) & (((abs(df1['C_DnShadow'] - df1['C_UpShadow']) / (df1['C_UpShadow'] * 100))
                                 < C_ShadowEqualsPercent))  , True, False)
    
    df1['C_IsDojiBody'] = np.where(( (df1['C_Range'] > 0)  & (df1['C_Body'] <= (df1['C_Range'] * (C_DojiBodyPercent / 100))) ), True, False)
    
    df1['C_Doji'] = df1['C_IsDojiBody'] & df1['C_ShadowEquals']
    
    df1['hl2'] = (df['high'] + df1['low'])/2
 #marking piercing data points based on condition satisfaction
    df1['C_PiercingBullish'] = np.where( ( (df1['C_DownTrend'].shift(1) & df1['C_BlackBody'].shift(1) & df1['C_LongBody'].shift(1)) &
                                        ( (df1['C_WhiteBody'] & df1['open'] <= df1['low'].shift(1)) & 
                                         (df1['close'] > df1['C_BodyMiddle'].shift(1)) & 
                                         (df1['close'] < df1['open'].shift(1)) )  )  , "1", "0" )
    
    return df1

In [84]:
df1PiercingBullish =PiercingBullish(df1)
print(df1PiercingBullish['C_PiercingBullish'].value_counts())

0    10416
1      724
Name: C_PiercingBullish, dtype: int64


Rising Window is a two-candle bullish continuation pattern that forms during an uptrend. Both candles in the pattern can be of any type with the exception of the Four-Price Doji. The most important characteristic of the pattern is a price gap between the first candle's high and the second candle's low.

In [85]:
def RisingWindowsBullish(df):
    
    df1 = df.copy()
    
    df1['MA100'] = df.rolling(window=100,min_periods=1 ).mean()['close']
    df1['MA200'] = df.rolling(window=200,min_periods=1 ).mean()['close']
    df1['MA50'] = df.rolling(window=50,min_periods=1 ).mean()['close']
    C_DownTrend = True
    C_UpTrend = True
    df1['C_DownTrend'] = np.where((  (df1['close'] < df1['MA50']) & (df1['MA50'] < df1['MA200']) ), True, False)
    df1['C_UpTrend'] = np.where(( (df1['close'] > df1['MA50'])& (df1['MA50'] > df1['MA200']) ), True, False)
    C_Len = 14 #ema depth for bodyAvg
    C_ShadowPercent = 5.0 # size of shadows
    C_ShadowEqualsPercent = 100.0
    C_DojiBodyPercent = 5.0
    C_Factor = 2.0 # shows the number of times the shadow dominates the candlestick body
    df1['C_BodyHi'] = df1[['close','open']].max(axis=1)
    df1['C_BodyLo'] = df1[['close','open']].min(axis=1)
    df1['C_Body'] = df1['C_BodyHi'] - df1['C_BodyLo']
    
    df1['C_BodyAvg'] = pd.Series.ewm(df1['C_Body'], span=C_Len).mean()
    
    
    df1.loc[df1['C_BodyAvg']> df1['C_Body']  , 'C_SmallBody'] = True
    df1.loc[df1['C_BodyAvg']<= df1['C_Body']  , 'C_SmallBody'] = False

    df1.loc[df1['C_BodyAvg']< df1['C_Body']  , 'C_LongBody'] = True
    df1.loc[df1['C_BodyAvg']>= df1['C_Body']  , 'C_LongBody'] = False
    
    
    df1['C_UpShadow'] = df1['high'] - df1['C_BodyHi']
    df1['C_DnShadow'] = df1['C_BodyLo'] - df1['low']
    df1['C_HasUpShadow'] = np.where((df1.C_UpShadow  > (C_ShadowPercent / (100*df1.C_Body))  ),
                                                     True, False)
    df1['C_HasDnShadow'] = np.where((df1.C_DnShadow  > (C_ShadowPercent / (100*df1.C_Body))  ),
                                                     True, False)
    
    df1['C_WhiteBody'] = np.where((df1.open  < df1.close ),True, False)
    df1['C_BlackBody'] = np.where((df1.open  > df1.close ),True, False)
    
    df1['C_Range'] = df1['high']-df1['low']
    df1['C_BodyMiddle']= df1['C_Body'] / (2 + df1['C_BodyLo'])
    
    df1['C_ShadowEquals'] = np.where(  ((df1['C_UpShadow'] == df1['C_DnShadow']) | 
                                 (( abs(df1['C_UpShadow'] - df1['C_DnShadow']) / (df1['C_DnShadow'] * 100) ) )  < 
                                 C_ShadowEqualsPercent) & (((abs(df1['C_DnShadow'] - df1['C_UpShadow']) / (df1['C_UpShadow'] * 100))
                                 < C_ShadowEqualsPercent))  , True, False)
    
    df1['C_IsDojiBody'] = np.where(( (df1['C_Range'] > 0)  & (df1['C_Body'] <= (df1['C_Range'] * (C_DojiBodyPercent / 100))) ), True, False)
    
    df1['C_Doji'] = df1['C_IsDojiBody'] & df1['C_ShadowEquals']
    
    df1['hl2'] = (df['high'] + df1['low'])/2
 #marking rising window data points based on condition satisfaction
    df1['C_RisingWindowBullish'] = np.where( ( ( (df1['C_Range'] !=0) & (df1['C_Range'].shift(1)!=0 ) ) &
                                             df1['C_UpTrend'].shift(1) &
                                            df1['low'] > df1['high'].shift(1) )  , "1", "0" )
    
    return df1

In [86]:
df1RisingWindowsBullish =RisingWindowsBullish(df1)
print(df1RisingWindowsBullish['C_RisingWindowBullish'].value_counts())

0    11140
Name: C_RisingWindowBullish, dtype: int64


A shooting star is a bearish candlestick with a long upper shadow, little or no lower shadow, and a small real body near the low of the day. It appears after an uptrend.

In [87]:
def ShootingStarBearish(df):
    
    df1 = df.copy()
    
    df1['MA100'] = df.rolling(window=100,min_periods=1 ).mean()['close']
    df1['MA200'] = df.rolling(window=200,min_periods=1 ).mean()['close']
    df1['MA50'] = df.rolling(window=50,min_periods=1 ).mean()['close']
    C_DownTrend = True
    C_UpTrend = True
    df1['C_DownTrend'] = np.where((  (df1['close'] < df1['MA50']) & (df1['MA50'] < df1['MA200']) ), True, False)
    df1['C_UpTrend'] = np.where(( (df1['close'] > df1['MA50'])& (df1['MA50'] > df1['MA200']) ), True, False)
    C_Len = 14 #ema depth for bodyAvg
    C_ShadowPercent = 5.0 # size of shadows
    C_ShadowEqualsPercent = 100.0
    C_DojiBodyPercent = 5.0
    C_Factor = 2.0 # shows the number of times the shadow dominates the candlestick body
    df1['C_BodyHi'] = df1[['close','open']].max(axis=1)
    df1['C_BodyLo'] = df1[['close','open']].min(axis=1)
    df1['C_Body'] = df1['C_BodyHi'] - df1['C_BodyLo']
    
    df1['C_BodyAvg'] = pd.Series.ewm(df1['C_Body'], span=C_Len).mean()
    
    
    df1.loc[df1['C_BodyAvg']> df1['C_Body']  , 'C_SmallBody'] = True
    df1.loc[df1['C_BodyAvg']<= df1['C_Body']  , 'C_SmallBody'] = False

    df1.loc[df1['C_BodyAvg']< df1['C_Body']  , 'C_LongBody'] = True
    df1.loc[df1['C_BodyAvg']>= df1['C_Body']  , 'C_LongBody'] = False
    
    
    df1['C_UpShadow'] = df1['high'] - df1['C_BodyHi']
    df1['C_DnShadow'] = df1['C_BodyLo'] - df1['low']
    df1['C_HasUpShadow'] = np.where((df1.C_UpShadow  > (C_ShadowPercent / (100*df1.C_Body))  ),
                                                     True, False)
    df1['C_HasDnShadow'] = np.where((df1.C_DnShadow  > (C_ShadowPercent / (100*df1.C_Body))  ),
                                                     True, False)
    
    df1['C_WhiteBody'] = np.where((df1.open  < df1.close ),True, False)
    df1['C_BlackBody'] = np.where((df1.open  > df1.close ),True, False)
    
    df1['C_Range'] = df1['high']-df1['low']
    df1['C_BodyMiddle']= df1['C_Body'] / (2 + df1['C_BodyLo'])
    
    df1['C_ShadowEquals'] = np.where(  ((df1['C_UpShadow'] == df1['C_DnShadow']) | 
                                 (( abs(df1['C_UpShadow'] - df1['C_DnShadow']) / (df1['C_DnShadow'] * 100) ) )  < 
                                 C_ShadowEqualsPercent) & (((abs(df1['C_DnShadow'] - df1['C_UpShadow']) / (df1['C_UpShadow'] * 100))
                                 < C_ShadowEqualsPercent))  , True, False)
    
    df1['C_IsDojiBody'] = np.where(( (df1['C_Range'] > 0)  & (df1['C_Body'] <= (df1['C_Range'] * (C_DojiBodyPercent / 100))) ), True, False)
    
    df1['C_Doji'] = df1['C_IsDojiBody'] & df1['C_ShadowEquals']
    
    df1['hl2'] = (df['high'] + df1['low'])/2
 #marking shooting star data points based on condition satisfaction
    df1['C_ShootingStarBearish'] = np.where( (df1['C_SmallBody'] & (df1['C_Body'] > 0 ) &
                                              (df1['C_BodyHi'] < df1['hl2']) &
                                             (df1['C_UpShadow'] >= C_Factor * df1['C_Body']) &
                                            (~ df1['C_HasDnShadow']) )  , "1", "0" )
    
    return df1

In [88]:
df1ShootingStarBearish =ShootingStarBearish(df1)
print(df1ShootingStarBearish['C_ShootingStarBearish'].value_counts())

0    10199
1      941
Name: C_ShootingStarBearish, dtype: int64



Black spinning tops are candlestick lines that are small, red-bodied, and possess shadows (upper and lower) that end up exceeding the length of candle bodies. They often signal indecision.
spinning top black conditions:

black body
at least one shadow required
body cannot be larger than any of the shadows
appears as a short line
if none of the shadows exceeds three times the body, it is allowed to appear as a long line

In [89]:
def SpinningTopBlack(df):
    
    df1 = df.copy()
    
    df1['MA100'] = df.rolling(window=100,min_periods=1 ).mean()['close']
    df1['MA200'] = df.rolling(window=200,min_periods=1 ).mean()['close']
    df1['MA50'] = df.rolling(window=50,min_periods=1 ).mean()['close']
    C_DownTrend = True
    C_UpTrend = True
    df1['C_DownTrend'] = np.where((  (df1['close'] < df1['MA50']) & (df1['MA50'] < df1['MA200']) ), True, False)
    df1['C_UpTrend'] = np.where(( (df1['close'] > df1['MA50'])& (df1['MA50'] > df1['MA200']) ), True, False)
    C_Len = 14 #ema depth for bodyAvg
    C_ShadowPercent = 5.0 # size of shadows
    C_ShadowEqualsPercent = 100.0
    C_DojiBodyPercent = 5.0
    C_Factor = 2.0 # shows the number of times the shadow dominates the candlestick body
    df1['C_BodyHi'] = df1[['close','open']].max(axis=1)
    df1['C_BodyLo'] = df1[['close','open']].min(axis=1)
    df1['C_Body'] = df1['C_BodyHi'] - df1['C_BodyLo']
    
    df1['C_BodyAvg'] = pd.Series.ewm(df1['C_Body'], span=C_Len).mean()
    
    
    df1.loc[df1['C_BodyAvg']> df1['C_Body']  , 'C_SmallBody'] = True
    df1.loc[df1['C_BodyAvg']<= df1['C_Body']  , 'C_SmallBody'] = False

    df1.loc[df1['C_BodyAvg']< df1['C_Body']  , 'C_LongBody'] = True
    df1.loc[df1['C_BodyAvg']>= df1['C_Body']  , 'C_LongBody'] = False
    
    
    df1['C_UpShadow'] = df1['high'] - df1['C_BodyHi']
    df1['C_DnShadow'] = df1['C_BodyLo'] - df1['low']
    df1['C_HasUpShadow'] = np.where((df1.C_UpShadow  > (C_ShadowPercent / (100*df1.C_Body))  ),
                                                     True, False)
    df1['C_HasDnShadow'] = np.where((df1.C_DnShadow  > (C_ShadowPercent / (100*df1.C_Body))  ),
                                                     True, False)
    
    df1['C_WhiteBody'] = np.where((df1.open  < df1.close ),True, False)
    df1['C_BlackBody'] = np.where((df1.open  > df1.close ),True, False)
    
    df1['C_Range'] = df1['high']-df1['low']
    df1['C_BodyMiddle']= df1['C_Body'] / (2 + df1['C_BodyLo'])
    
    df1['C_ShadowEquals'] = np.where(  ((df1['C_UpShadow'] == df1['C_DnShadow']) | 
                                 (( abs(df1['C_UpShadow'] - df1['C_DnShadow']) / (df1['C_DnShadow'] * 100) ) )  < 
                                 C_ShadowEqualsPercent) & (((abs(df1['C_DnShadow'] - df1['C_UpShadow']) / (df1['C_UpShadow'] * 100))
                                 < C_ShadowEqualsPercent))  , True, False)
    
    df1['C_IsDojiBody'] = np.where(( (df1['C_Range'] > 0)  & (df1['C_Body'] <= (df1['C_Range'] * (C_DojiBodyPercent / 100))) ), True, False)
    
    df1['C_Doji'] = df1['C_IsDojiBody'] & df1['C_ShadowEquals']
    
    df1['hl2'] = (df['high'] + df1['low'])/2
    
    C_SpinningTopBlackPercent = 34.0
    
    
    
    
    df1['C_IsSpinningTop'] = np.where( (( df1['C_DnShadow'] >= (df1['C_Range'] / (100 * C_SpinningTopBlackPercent)) ) &
                                      (df1['C_UpShadow'] >= (df1['C_Range'] / (100 * C_SpinningTopBlackPercent)) ) &
                                     (~df1['C_IsDojiBody'] ) )  , True, False )
   
 #marking spinning top black  data points based on condition satisfaction
    df1['C_SpinningTopBlack'] = np.where((df1['C_IsSpinningTop'] & df1['C_BlackBody']) , "1", "0")
    
    
    return df1

In [90]:
df1SpinningTopBlack =SpinningTopBlack(df1)
print(df1SpinningTopBlack['C_SpinningTopBlack'].value_counts())

0    7801
1    3339
Name: C_SpinningTopBlack, dtype: int64


White spinning tops are candlestick lines that are small, green-bodied, and possess shadows (upper and lower) that end up exceeding the length of candle bodies. They often signal indecision between buyer and seller.


white spinning top conditions:
Construction:

white body
at least one shadow required
at least one shadow has to be longer than the body
appears on as a long and a short line
if appears on as a long line, none of the shadows can exceed three times the body (otherwise we would have the High Wave basic candle)

In [91]:
def SpinningTopWhite(df):
    
    df1 = df.copy()
    
    df1['MA100'] = df.rolling(window=100,min_periods=1 ).mean()['close']
    df1['MA200'] = df.rolling(window=200,min_periods=1 ).mean()['close']
    df1['MA50'] = df.rolling(window=50,min_periods=1 ).mean()['close']
    C_DownTrend = True
    C_UpTrend = True
    df1['C_DownTrend'] = np.where((  (df1['close'] < df1['MA50']) & (df1['MA50'] < df1['MA200']) ), True, False)
    df1['C_UpTrend'] = np.where(( (df1['close'] > df1['MA50'])& (df1['MA50'] > df1['MA200']) ), True, False)
    C_Len = 14 #ema depth for bodyAvg
    C_ShadowPercent = 5.0 # size of shadows
    C_ShadowEqualsPercent = 100.0
    C_DojiBodyPercent = 5.0
    C_Factor = 2.0 # shows the number of times the shadow dominates the candlestick body
    df1['C_BodyHi'] = df1[['close','open']].max(axis=1)
    df1['C_BodyLo'] = df1[['close','open']].min(axis=1)
    df1['C_Body'] = df1['C_BodyHi'] - df1['C_BodyLo']
    
    df1['C_BodyAvg'] = pd.Series.ewm(df1['C_Body'], span=C_Len).mean()
    
    
    df1.loc[df1['C_BodyAvg']> df1['C_Body']  , 'C_SmallBody'] = True
    df1.loc[df1['C_BodyAvg']<= df1['C_Body']  , 'C_SmallBody'] = False

    df1.loc[df1['C_BodyAvg']< df1['C_Body']  , 'C_LongBody'] = True
    df1.loc[df1['C_BodyAvg']>= df1['C_Body']  , 'C_LongBody'] = False
    
    
    df1['C_UpShadow'] = df1['high'] - df1['C_BodyHi']
    df1['C_DnShadow'] = df1['C_BodyLo'] - df1['low']
    df1['C_HasUpShadow'] = np.where((df1.C_UpShadow  > (C_ShadowPercent / (100*df1.C_Body))  ),
                                                     True, False)
    df1['C_HasDnShadow'] = np.where((df1.C_DnShadow  > (C_ShadowPercent / (100*df1.C_Body))  ),
                                                     True, False)
    
    df1['C_WhiteBody'] = np.where((df1.open  < df1.close ),True, False)
    df1['C_BlackBody'] = np.where((df1.open  > df1.close ),True, False)
    
    df1['C_Range'] = df1['high']-df1['low']
    df1['C_BodyMiddle']= df1['C_Body'] / (2 + df1['C_BodyLo'])
    
    df1['C_ShadowEquals'] = np.where(  ((df1['C_UpShadow'] == df1['C_DnShadow']) | 
                                 (( abs(df1['C_UpShadow'] - df1['C_DnShadow']) / (df1['C_DnShadow'] * 100) ) )  < 
                                 C_ShadowEqualsPercent) & (((abs(df1['C_DnShadow'] - df1['C_UpShadow']) / (df1['C_UpShadow'] * 100))
                                 < C_ShadowEqualsPercent))  , True, False)
    
    df1['C_IsDojiBody'] = np.where(( (df1['C_Range'] > 0)  & (df1['C_Body'] <= (df1['C_Range'] * (C_DojiBodyPercent / 100))) ), True, False)
    
    df1['C_Doji'] = df1['C_IsDojiBody'] & df1['C_ShadowEquals']
    
    df1['hl2'] = (df['high'] + df1['low'])/2
    
    C_SpinningTopWhitePercent = 34.0
    
    
    
    
    df1['C_IsSpinningTopWhite'] = np.where( (( df1['C_DnShadow'] >= (df1['C_Range'] / (100 * C_SpinningTopWhitePercent)) ) &
                                      (df1['C_UpShadow'] >= (df1['C_Range'] / (100 * C_SpinningTopWhitePercent)) ) &
                                     (~df1['C_IsDojiBody'] ) )  , True, False )
    #marking spinning top white  data points based on condition satisfaction

    df1['C_SpinningTopWhite'] = np.where((df1['C_IsSpinningTopWhite'] & df1['C_WhiteBody']) , "1", "0")
    
    
    return df1

In [92]:
df1SpinningTopWhite =SpinningTopWhite(df1)
print(df1SpinningTopWhite['C_SpinningTopWhite'].value_counts())

0    8086
1    3054
Name: C_SpinningTopWhite, dtype: int64


The Bearish Tri-Star pattern is a three-line bearish reversal pattern which all three lines are doji candles (any doji candle, except the Four-Price Doji).

Construction:

First candle
a doji candle in an uptrend
Second candle
a doji candle
a body above the prior body
Third candle
a doji candle
a body below the prior body

In [93]:
def TriStarBearish(df):
    
    df1 = df.copy()
    
    df1['MA100'] = df.rolling(window=100,min_periods=1 ).mean()['close']
    df1['MA200'] = df.rolling(window=200,min_periods=1 ).mean()['close']
    df1['MA50'] = df.rolling(window=50,min_periods=1 ).mean()['close']
    C_DownTrend = True
    C_UpTrend = True
    df1['C_DownTrend'] = np.where((  (df1['close'] < df1['MA50']) & (df1['MA50'] < df1['MA200']) ), True, False)
    df1['C_UpTrend'] = np.where(( (df1['close'] > df1['MA50'])& (df1['MA50'] > df1['MA200']) ), True, False)
    C_Len = 14 #ema depth for bodyAvg
    C_ShadowPercent = 5.0 # size of shadows
    C_ShadowEqualsPercent = 100.0
    C_DojiBodyPercent = 5.0
    C_Factor = 2.0 # shows the number of times the shadow dominates the candlestick body
    df1['C_BodyHi'] = df1[['close','open']].max(axis=1)
    df1['C_BodyLo'] = df1[['close','open']].min(axis=1)
    df1['C_Body'] = df1['C_BodyHi'] - df1['C_BodyLo']
    
    df1['C_BodyAvg'] = pd.Series.ewm(df1['C_Body'], span=C_Len).mean()
    
    
    df1.loc[df1['C_BodyAvg']> df1['C_Body']  , 'C_SmallBody'] = True
    df1.loc[df1['C_BodyAvg']<= df1['C_Body']  , 'C_SmallBody'] = False

    df1.loc[df1['C_BodyAvg']< df1['C_Body']  , 'C_LongBody'] = True
    df1.loc[df1['C_BodyAvg']>= df1['C_Body']  , 'C_LongBody'] = False
    
    
    df1['C_UpShadow'] = df1['high'] - df1['C_BodyHi']
    df1['C_DnShadow'] = df1['C_BodyLo'] - df1['low']
    df1['C_HasUpShadow'] = np.where((df1.C_UpShadow  > (C_ShadowPercent / (100*df1.C_Body))  ),
                                                     True, False)
    df1['C_HasDnShadow'] = np.where((df1.C_DnShadow  > (C_ShadowPercent / (100*df1.C_Body))  ),
                                                     True, False)
    
    df1['C_WhiteBody'] = np.where((df1.open  < df1.close ),True, False)
    df1['C_BlackBody'] = np.where((df1.open  > df1.close ),True, False)
    
    df1['C_Range'] = df1['high']-df1['low']
    df1['C_BodyMiddle']= df1['C_Body'] / (2 + df1['C_BodyLo'])
    
    df1['C_ShadowEquals'] = np.where(  ((df1['C_UpShadow'] == df1['C_DnShadow']) | 
                                 (( abs(df1['C_UpShadow'] - df1['C_DnShadow']) / (df1['C_DnShadow'] * 100) ) )  < 
                                 C_ShadowEqualsPercent) & (((abs(df1['C_DnShadow'] - df1['C_UpShadow']) / (df1['C_UpShadow'] * 100))
                                 < C_ShadowEqualsPercent))  , True, False)
    
    df1['C_IsDojiBody'] = np.where(( (df1['C_Range'] > 0)  & (df1['C_Body'] <= (df1['C_Range'] * (C_DojiBodyPercent / 100))) ), True, False)
    
    df1['C_Doji'] = df1['C_IsDojiBody'] & df1['C_ShadowEquals']
    
    df1['hl2'] = (df['high'] + df1['low'])/2
    
    
    
    
    df1['C_3Dojis'] = np.where( (df1['C_Doji'].shift(2) & df1['C_Doji'].shift(1) & df1['C_Doji'] )  , True, False )
    df1['C_BodyGapUp'] = np.where( (df1['C_BodyHi'].shift(1) < df1['C_BodyLo'] )  , True, False )
    df1['C_BodyGapDn'] = np.where( (df1['C_BodyLo'].shift(1) > df1['C_BodyHi'] )  , True, False )
   
  #marking tristar bearish data points based on condition satisfaction
    df1['C_TriStarBearish'] = np.where(( df1['C_3Dojis'] &  df1['C_UpTrend'].shift(2) &
                                       df1['C_BodyGapUp'].shift(1) & df1['C_BodyGapDn'] ) , "1", "0")
    
    
    return df1

In [94]:
df1TriStarBearish =TriStarBearish(df1)
print(df1TriStarBearish['C_TriStarBearish'].value_counts())

0    11140
Name: C_TriStarBearish, dtype: int64


The Bullish Tri-Star pattern is a three-line bullish reversal pattern which all three lines are doji candles (any doji candle, except the Four-Price Doji).
Construction:

First candle
a doji candle in a downtrend
Second candle
a doji candle
a body below the prior body
Third candle
a doji candle
a body above the prior body

In [95]:
def TriStarBullish(df):
    
    df1 = df.copy()
    
    df1['MA100'] = df.rolling(window=100,min_periods=1 ).mean()['close']
    df1['MA200'] = df.rolling(window=200,min_periods=1 ).mean()['close']
    df1['MA50'] = df.rolling(window=50,min_periods=1 ).mean()['close']
    C_DownTrend = True
    C_UpTrend = True
    df1['C_DownTrend'] = np.where((  (df1['close'] < df1['MA50']) & (df1['MA50'] < df1['MA200']) ), True, False)
    df1['C_UpTrend'] = np.where(( (df1['close'] > df1['MA50'])& (df1['MA50'] > df1['MA200']) ), True, False)
    C_Len = 14 #ema depth for bodyAvg
    C_ShadowPercent = 5.0 # size of shadows
    C_ShadowEqualsPercent = 100.0
    C_DojiBodyPercent = 5.0
    C_Factor = 2.0 # shows the number of times the shadow dominates the candlestick body
    df1['C_BodyHi'] = df1[['close','open']].max(axis=1)
    df1['C_BodyLo'] = df1[['close','open']].min(axis=1)
    df1['C_Body'] = df1['C_BodyHi'] - df1['C_BodyLo']
    
    df1['C_BodyAvg'] = pd.Series.ewm(df1['C_Body'], span=C_Len).mean()
    
    
    df1.loc[df1['C_BodyAvg']> df1['C_Body']  , 'C_SmallBody'] = True
    df1.loc[df1['C_BodyAvg']<= df1['C_Body']  , 'C_SmallBody'] = False

    df1.loc[df1['C_BodyAvg']< df1['C_Body']  , 'C_LongBody'] = True
    df1.loc[df1['C_BodyAvg']>= df1['C_Body']  , 'C_LongBody'] = False
    
    
    df1['C_UpShadow'] = df1['high'] - df1['C_BodyHi']
    df1['C_DnShadow'] = df1['C_BodyLo'] - df1['low']
    df1['C_HasUpShadow'] = np.where((df1.C_UpShadow  > (C_ShadowPercent / (100*df1.C_Body))  ),
                                                     True, False)
    df1['C_HasDnShadow'] = np.where((df1.C_DnShadow  > (C_ShadowPercent / (100*df1.C_Body))  ),
                                                     True, False)
    
    df1['C_WhiteBody'] = np.where((df1.open  < df1.close ),True, False)
    df1['C_BlackBody'] = np.where((df1.open  > df1.close ),True, False)
    
    df1['C_Range'] = df1['high']-df1['low']
    df1['C_BodyMiddle']= df1['C_Body'] / (2 + df1['C_BodyLo'])
    
    df1['C_ShadowEquals'] = np.where(  ((df1['C_UpShadow'] == df1['C_DnShadow']) | 
                                 (( abs(df1['C_UpShadow'] - df1['C_DnShadow']) / (df1['C_DnShadow'] * 100) ) )  < 
                                 C_ShadowEqualsPercent) & (((abs(df1['C_DnShadow'] - df1['C_UpShadow']) / (df1['C_UpShadow'] * 100))
                                 < C_ShadowEqualsPercent))  , True, False)
    
    df1['C_IsDojiBody'] = np.where(( (df1['C_Range'] > 0)  & (df1['C_Body'] <= (df1['C_Range'] * (C_DojiBodyPercent / 100))) ), True, False)
    
    df1['C_Doji'] = df1['C_IsDojiBody'] & df1['C_ShadowEquals']
    
    df1['hl2'] = (df['high'] + df1['low'])/2
    
    
    
    df1['C_3Dojis'] = np.where( (df1['C_Doji'].shift(2) & df1['C_Doji'].shift(1) & df1['C_Doji'] )  , True, False )
    df1['C_BodyGapUp'] = np.where( (df1['C_BodyHi'].shift(1) < df1['C_BodyLo'] )  , True, False )
    df1['C_BodyGapDn'] = np.where( (df1['C_BodyLo'].shift(1) > df1['C_BodyHi'] )  , True, False )
   
#marking tristar bullish data points based on condition satisfaction
    df1['C_TriStarBullish'] = np.where(( df1['C_3Dojis'] &  df1['C_DownTrend'].shift(2) &
                                       df1['C_BodyGapDn'].shift(1) & df1['C_BodyGapUp'] ) , "1", "0")
    
    
    return df1

In [96]:
df1TriStarBullish =TriStarBullish(df1)
print(df1TriStarBullish['C_TriStarBullish'].value_counts())

0    11140
Name: C_TriStarBullish, dtype: int64


The bullish reversal candlestick pattern known as the Tweezer Bottom forms at the bottom of a downtrend. It is made up of two candlesticks, the first of which is bearish and the second of which is bullish.

In [97]:
def TweezerBottomBullish(df):
    
    df1 = df.copy()
    
    df1['MA100'] = df.rolling(window=100,min_periods=1 ).mean()['close']
    df1['MA200'] = df.rolling(window=200,min_periods=1 ).mean()['close']
    df1['MA50'] = df.rolling(window=50,min_periods=1 ).mean()['close']
    C_DownTrend = True
    C_UpTrend = True
    df1['C_DownTrend'] = np.where((  (df1['close'] < df1['MA50']) & (df1['MA50'] < df1['MA200']) ), True, False)
    df1['C_UpTrend'] = np.where(( (df1['close'] > df1['MA50'])& (df1['MA50'] > df1['MA200']) ), True, False)
    C_Len = 14 #ema depth for bodyAvg
    C_ShadowPercent = 5.0 # size of shadows
    C_ShadowEqualsPercent = 100.0
    C_DojiBodyPercent = 5.0
    C_Factor = 2.0 # shows the number of times the shadow dominates the candlestick body
    df1['C_BodyHi'] = df1[['close','open']].max(axis=1)
    df1['C_BodyLo'] = df1[['close','open']].min(axis=1)
    df1['C_Body'] = df1['C_BodyHi'] - df1['C_BodyLo']
    
    df1['C_BodyAvg'] = pd.Series.ewm(df1['C_Body'], span=C_Len).mean()
    
    
    df1.loc[df1['C_BodyAvg']> df1['C_Body']  , 'C_SmallBody'] = True
    df1.loc[df1['C_BodyAvg']<= df1['C_Body']  , 'C_SmallBody'] = False

    df1.loc[df1['C_BodyAvg']< df1['C_Body']  , 'C_LongBody'] = True
    df1.loc[df1['C_BodyAvg']>= df1['C_Body']  , 'C_LongBody'] = False
    
    
    df1['C_UpShadow'] = df1['high'] - df1['C_BodyHi']
    df1['C_DnShadow'] = df1['C_BodyLo'] - df1['low']
    df1['C_HasUpShadow'] = np.where((df1.C_UpShadow  > (C_ShadowPercent / (100*df1.C_Body))  ),
                                                     True, False)
    df1['C_HasDnShadow'] = np.where((df1.C_DnShadow  > (C_ShadowPercent / (100*df1.C_Body))  ),
                                                     True, False)
    
    df1['C_WhiteBody'] = np.where((df1.open  < df1.close ),True, False)
    df1['C_BlackBody'] = np.where((df1.open  > df1.close ),True, False)
    
    df1['C_Range'] = df1['high']-df1['low']
    df1['C_BodyMiddle']= df1['C_Body'] / (2 + df1['C_BodyLo'])
    
    df1['C_ShadowEquals'] = np.where(  ((df1['C_UpShadow'] == df1['C_DnShadow']) | 
                                 (( abs(df1['C_UpShadow'] - df1['C_DnShadow']) / (df1['C_DnShadow'] * 100) ) )  < 
                                 C_ShadowEqualsPercent) & (((abs(df1['C_DnShadow'] - df1['C_UpShadow']) / (df1['C_UpShadow'] * 100))
                                 < C_ShadowEqualsPercent))  , True, False)
    
    df1['C_IsDojiBody'] = np.where(( (df1['C_Range'] > 0)  & (df1['C_Body'] <= (df1['C_Range'] * (C_DojiBodyPercent / 100))) ), True, False)
    
    df1['C_Doji'] = df1['C_IsDojiBody'] & df1['C_ShadowEquals']
    
    df1['hl2'] = (df['high'] + df1['low'])/2
    
    

#marking tweezer bottom bullish points based on condition satisfaction
    df1['C_TweezerBottomBullish'] = np.where(( df1['C_UpTrend'].shift(1) &
                                           ( (~df1['C_IsDojiBody']) | (df1['C_HasUpShadow'] & df1['C_HasDnShadow']) )  &
                                            (abs(df1['low']-df1['low'].shift(1)) <= (df1['C_BodyAvg']*0.05) ) &
                                            df1['C_BlackBody'].shift(1) & df1['C_WhiteBody'] &  df1['C_LongBody'].shift(1) ) , "1", "0")
    
    
    return df1

In [98]:
df1TweezerBottomBullish =TweezerBottomBullish(df1)
print(df1TweezerBottomBullish['C_TweezerBottomBullish'].value_counts())

0    11095
1       45
Name: C_TweezerBottomBullish, dtype: int64


A bearish reversal candlestick pattern known as the Tweezer Top forms at the peak of an uptrend. It is made up of two candlesticks, the first of which is bullish and the second of which is bearish. Both tweezer candlesticks reach a nearly identical high.

In [99]:
def TweezerTopBearish(df):
    
    df1 = df.copy()
    
    df1['MA100'] = df.rolling(window=100,min_periods=1 ).mean()['close']
    df1['MA200'] = df.rolling(window=200,min_periods=1 ).mean()['close']
    df1['MA50'] = df.rolling(window=50,min_periods=1 ).mean()['close']
    C_DownTrend = True
    C_UpTrend = True
    df1['C_DownTrend'] = np.where((  (df1['close'] < df1['MA50']) & (df1['MA50'] < df1['MA200']) ), True, False)
    df1['C_UpTrend'] = np.where(( (df1['close'] > df1['MA50'])& (df1['MA50'] > df1['MA200']) ), True, False)
    C_Len = 14 #ema depth for bodyAvg
    C_ShadowPercent = 5.0 # size of shadows
    C_ShadowEqualsPercent = 100.0
    C_DojiBodyPercent = 5.0
    C_Factor = 2.0 # shows the number of times the shadow dominates the candlestick body
    df1['C_BodyHi'] = df1[['close','open']].max(axis=1)
    df1['C_BodyLo'] = df1[['close','open']].min(axis=1)
    df1['C_Body'] = df1['C_BodyHi'] - df1['C_BodyLo']
    
    df1['C_BodyAvg'] = pd.Series.ewm(df1['C_Body'], span=C_Len).mean()
    
    
    df1.loc[df1['C_BodyAvg']> df1['C_Body']  , 'C_SmallBody'] = True
    df1.loc[df1['C_BodyAvg']<= df1['C_Body']  , 'C_SmallBody'] = False

    df1.loc[df1['C_BodyAvg']< df1['C_Body']  , 'C_LongBody'] = True
    df1.loc[df1['C_BodyAvg']>= df1['C_Body']  , 'C_LongBody'] = False
    
    
    df1['C_UpShadow'] = df1['high'] - df1['C_BodyHi']
    df1['C_DnShadow'] = df1['C_BodyLo'] - df1['low']
    df1['C_HasUpShadow'] = np.where((df1.C_UpShadow  > (C_ShadowPercent / (100*df1.C_Body))  ),
                                                     True, False)
    df1['C_HasDnShadow'] = np.where((df1.C_DnShadow  > (C_ShadowPercent / (100*df1.C_Body))  ),
                                                     True, False)
    
    df1['C_WhiteBody'] = np.where((df1.open  < df1.close ),True, False)
    df1['C_BlackBody'] = np.where((df1.open  > df1.close ),True, False)
    
    df1['C_Range'] = df1['high']-df1['low']
    df1['C_BodyMiddle']= df1['C_Body'] / (2 + df1['C_BodyLo'])
    
    df1['C_ShadowEquals'] = np.where(  ((df1['C_UpShadow'] == df1['C_DnShadow']) | 
                                 (( abs(df1['C_UpShadow'] - df1['C_DnShadow']) / (df1['C_DnShadow'] * 100) ) )  < 
                                 C_ShadowEqualsPercent) & (((abs(df1['C_DnShadow'] - df1['C_UpShadow']) / (df1['C_UpShadow'] * 100))
                                 < C_ShadowEqualsPercent))  , True, False)
    
    df1['C_IsDojiBody'] = np.where(( (df1['C_Range'] > 0)  & (df1['C_Body'] <= (df1['C_Range'] * (C_DojiBodyPercent / 100))) ), True, False)
    
    df1['C_Doji'] = df1['C_IsDojiBody'] & df1['C_ShadowEquals']
    
    df1['hl2'] = (df['high'] + df1['low'])/2
    
    
#marking tweezer top bearish data points based on condition satisfaction

    df1['C_TweezerTopBearish'] = np.where(( df1['C_UpTrend'].shift(1) &
                                           ( (~df1['C_IsDojiBody']) | (df1['C_HasUpShadow'] & df1['C_HasDnShadow']) )  &
                                            (abs(df1['high']-df1['high'].shift(1)) <= (df1['C_BodyAvg']*0.05) ) &
                                            df1['C_WhiteBody'].shift(1) & df1['C_BlackBody'] &  df1['C_LongBody'].shift(1) ) , "1", "0")
    
    
    return df1

In [100]:
df1TweezerTopBearish =TweezerTopBearish(df1)
print(df1TweezerTopBearish['C_TweezerTopBearish'].value_counts())

0    11089
1       51
Name: C_TweezerTopBearish, dtype: int64
